# Research data supporting "Reliable Insights into Surface Chemistry Enabled by Combining Experiments with Fast and Accurate Theory"

This notebook accompanies our paper: **Reliable Insights into Surface Chemistry Enabled by Combining Experiments with Fast and Accurate Theory**. It can be found on GitHub at https://github.com/benshi97/Data_Atomistic_Insights and explored interactively on [Colab](https://colab.research.google.com/github/benshi97/Data_Atomistic_Insights/blob/main/analyse.ipynb).

### Abstract

<!-- The adsorption energy of a molecule onto the surface of a material underpins a wide array of applications, spanning heterogeneous catalysis, gas storage and many more. It is the key quantity where experimental measurements and theoretical calculations meet, with agreement being necessary for reliable predictions of reaction rates and mechanisms. The prototypical molecule-surface system is CO adsorbed on MgO, but despite intense scrutiny from theory and experiment, there is still no consensus on its adsorption energy. In particular, the large cost of accurate many-body methods makes reaching converged theoretical estimates difficult, generating a wide range of values. In this work, we address this challenge, leveraging the latest advances in diffusion Monte Carlo (DMC) and coupled cluster theory [CCSD(T)], to obtain accurate predictions for CO on MgO. These reliable theoretical estimates allow us to evaluate the inconsistencies in published temperature programmed desorption experiments, revealing that they arise from variations in employed pre-exponential factors. Using this insight, we derive new experimental estimates of the (electronic) adsorption energy from a more precise pre-exponential factor. With this effort, we are able to reach consensus between multiple theoretical calculations and multiple experiments for the first time. In addition, we show that our recently developed cluster-based CCSD(T) approach provides a low cost route towards achieving accurate adsorption energies. This sets the stage for affordable and reliable theoretical predictions of reaction mechanisms and rates to guide the realization of new catalysts and gas storage materials. -->


## Contents
* [Table S1 and Figure 1 - Past CO on MgO *E*<sub>ads</sub> and Current Work](#tables1fig1)
* [Table S2 and S3 - Validating the revPBE-D4 geometry and Computing its $\Delta_\textrm{geom}$](#tables2s3)
* [Table S4 and S5 - Periodic CCSD(T) Convergence](#tables4)
* [Table S6 - Periodic DMC Convergence](#tables6)
* [Table S7 - DFT Convergence](#tables7)
* [Table S9 to S11 and Figure 2 - SKZCAM Cluster CCSD(T) Convergence](#tables9-s11)
* [Table S8 - Final periodic CCSD(T), periodic DMC and cluster CCSD(T) $E_\textrm{ads}$ and their individual contributions](#tables8)
* [Table S12 - Analysis of previous computational work on CO on MgO](#tables12)
* [Table S13 - Computation of zero-point energy and thermal contribution terms to convert $H_\textrm{ads}$ to $E_\textrm{ads}$](#tables13)
* [Figure 3 and S1 - Converting previous experiment $H_\textrm{ads}$ or $E_\textrm{act}$ (for TPD) to $E_\textrm{ads}$](#fig3)
* [Table S14 - Final best estimate of TPD experimental adsorption energies](#tables14)
* [Table S15 - Effect of CO coverage on $E_\textrm{ads}$](#tables15)


In [79]:
# Import necessary functions
from Scripts.analysis_scripts import *
from Scripts.plot_scripts import *


# Check if we are in Google Colab environment
try:
    import google.colab
    IN_COLAB = True
    usetex = False
    texfalse_import()

except:
    import os
    IN_COLAB = False
    if os.path.expanduser('~') == '/home/shixubenjamin':
        usetex = True
        textrue_import()
    else:
        usetex = False    
        texfalse_import()

# If in Google Colab, install the necessary data and set up the necessary environment
if IN_COLAB == True:
    !rm -rf /content/Data_CO_on_MgO-main /content/main.zip
    !wget https://github.com/benshi97/Data_CO_on_MgO/archive/refs/heads/main.zip
    !unzip /content/main.zip
    ! apt install ase
    !pip install pyblock datetime
    %pwd
    %cd /content/Data_CO_on_MgO-main

In [80]:
# Define the systems to be analysed

# The ensemble of DFT exchange-correlation functionals used according to crystal system
crystal_xc_func_ensemble = {
    'MgO': ["01_PBE-D2-Ne", "02_revPBE-D4", "03_vdW-DF", "04_rev-vdW-DF2", "05_PBE0-D4", "06_B3LYP-D2-Ne"],
    'r-TiO2': ["01_PBE-U-TSHI", "02_revPBE-D4", "03_vdW-DF", "04_rev-vdW-DF2", "05_R2SCAN-rVV10", "06_HSE06-D4"],
    'a-TiO2': ["01_PBE-U-TSHI", "02_revPBE-D4", "03_vdW-DF", "04_rev-vdW-DF2", "05_R2SCAN-rVV10", "06_HSE06-D4"]
}

# All the molecule-surface systems to be analysed
molecule_surface_systems = {
    'MgO': ['CH4', 'CH4 Monolayer', 'C2H6', 'C2H6 Monolayer', 'CO', 'C6H6', 'N2O Parallel', 'N2O Tilted', 'NO Vertical-Hollow', 'NO Vertical-Mg', 'NO Bent-Bridge', 'NO Bent-Mg', 'NO Bent-O', 'NO Dimer', 'H2O Monomer', 'H2O Tetramer', 'CH3OH Tilted', 'CH3OH Parallel', 'CH3OH Tetramer', 'NH3', 'CO2 Physisorbed', 'CO2 Chemisorbed'],
    'MgO_8x8': ['C6H6', 'H2O Tetramer', 'CH3OH Tetramer'], # Systems that use a 8x8 MgO supercell rather than the default 4x4
    'r-TiO2': ['CH4','CO2 Parallel','CO2 Tilted','H2O','CH3OH'],
    'a-TiO2': ['H2O','NH3']
}

# A dictionary which converts the DFT functional, surface and adsorbate dictionary labels to LaTeX formatted labels
convert_to_nice_labels = {
    'xc_functionals': {
        '01_PBE-D2-Ne': 'PBE-D2[Ne]',
        '02_revPBE-D4': 'revPBE-D4',
        '03_vdW-DF': 'vdW-DF',
        '04_rev-vdW-DF2': 'rev-vdW-DF2',
        '05_PBE0-D4': 'PBE0-D4',
        '06_B3LYP-D2-Ne': 'B3LYP-D2[Ne]',
        '01_PBE-U-TSHI': 'PBE+U-TS/HI',
        '05_R2SCAN-rVV10': r'r$^2$SCAN-rVV10',
        '06_HSE06-D4': 'HSE06-D4'
    },
    'surface': {
        'MgO': 'MgO (001)',
        'r-TiO2': r'\ce{TiO2} rutile (110)',
        'a-TiO2': r'\ce{TiO2} anatase (101)'
    },
    'adsorbates': {
        surface: {
            molecule: r'\ce{' + molecule.split()[0] + '}'
            if len(molecule.split()) == 1 else
            molecule.split()[1] + r' \ce{' + molecule.split()[0] + '}'
            for molecule in molecules
        }
        for surface, molecules in molecule_surface_systems.items()
    }
}

# *H*<sub>ads</sub> from the experimental literature

In [81]:
# Convert 'Experimental_Data.csv' csv file to pandas DataFrame

expt_hads_df = pd.read_csv('Data/Miscellaneous/Experimental_Data.csv',dtype=object)
expt_hads_df = expt_hads_df.reset_index(drop=True)

# Convert to latex table
convert_df_to_latex_input(
df = expt_hads_df,
start_input = '\\begin{turnpage}\n\\begin{table}\n',
end_input = '\n\\end{table}\n\\end{turnpage}',
label = "tab:expt_hads",
caption = "Experimental adsorption enthalpies for the systems studied within this work.",
replace_input = {
    "+-": r"$\pm$"
},
adjustbox = 1.6,
column_format = "lrrrrrrp{12cm}",
index = False,
filename = 'Tables/SI_Table-Experimental_Hads.tex')

expt_and_autoskzcam_hads_df = expt_hads_df.copy()

expt_hads_df.columns = ['Surface','Molecule','Temperature','log(ν)', 'Eact', 'Hads', 'Error', 'Details']
expt_hads_df = expt_hads_df.replace({
    r'\\ce\{|\}|\(001\)|\(110\)|\(101\)|\(\)': '',
    r'\$\\to\$': '->',
    r'\+-': '±',
}, regex=True)

expt_hads_df = expt_hads_df.applymap(lambda x: x.strip() if isinstance(x, str) else x)
expt_hads_df['Surface'] = expt_hads_df['Surface'].replace({'TiO2 rutile': 'r-TiO2', 'TiO2 anatase': 'a-TiO2'})
expt_hads_df

/tmp/ipykernel_720/3524727339.py:30: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  expt_hads_df = expt_hads_df.applymap(lambda x: x.strip() if isinstance(x, str) else x)


,Surface,Molecule,Temperature,log(ν),Eact,Hads,Error,Details
0,MgO,CH4,47,13.1 ± 2.0,-115,-113,19,Dilute limit $E_\textrm{act$ estimate by Tait ...
1,MgO,C2H6,75,14.9 ± 2.0,-221,-218,30,Dilute limit $E_\textrm{act$ estimate by Tait ...
2,MgO,CO,61,13.8 ± 1.6,-,-179,19,Average of the $H_\textrm{ads$ re-analysis by ...
3,MgO,N2O,77,13.0 -> 14.0 ± 2.0,-230,-242,31,$E_\textrm{act$ measured by Lian \etal{~\cite{...
4,MgO,C6H6,162,15.1 ± 1.6,-,-488,68,Average taken of $H_\textrm{ads$ re-analyzed b...
5,MgO,H2O,203,-,-,-520,121,$H_\textrm{ads$ from Ferry \etal{~\cite{alessi...
6,MgO,NH3,160,13.0 -> 14.0 ± 2.0,-581,-606,63,$E_\textrm{act$ measurement from Arthur \etal{...
7,MgO,Physisorbed CO2,120,13.0 -> 14.0 ± 2.0,-408,-426,48,$E_\textrm{act$ measurement from Meixner \etal...
8,MgO,Chemisorbed CO2,230,13.0 -> 14.0 ± 2.0,-619,-654,123,$E_\textrm{act$ measurement from Chakradhar an...
9,MgO,Monolayer CH4,47,13.1 ± 2.0,-131,-129,19,Monolayer $E_\textrm{act$ estimate by Tait \et...


# Computing *E*<sub>rlx</sub>, *E*<sub>therm</sub> and *E*<sub>ZPV</sub> from DFT

### SI - DFT Parameters

In [82]:
dft_parameters_dict = {
    'MgO': {
        '(meta)GGA': {
            'Supercell Size': '4x4' ,
            'Number of Layers': '4(2)' ,
            'K-point Mesh': '2x2x1',
            'Energy cutoff': '600',
            'Vacuum': '15',
            'Metal PAW potential': 'Mg_pv (4e core)',
        },
        'hybrid': {
            'Supercell Size': '4x4' ,
            'Number of Layers': '4(2)' ,
            'K-point Mesh': '2x2x1(2x2x1)',
            'Energy cutoff': '600',
            'Vacuum': '15',
            'Metal PAW potential': 'Mg_pv (4e core)',
        }
    },
    'r-TiO2': {
        '(meta)GGA': {
            'Supercell Size': 'p(4x2)' ,
            'Number of Layers': '5(3) O-Ti-O' ,
            'K-point Mesh': '2x2x1',
            'Energy cutoff': '600',
            'Vacuum': '15',
            'Metal PAW potential': 'Ti_pv (12e core)',
        },
        'hybrid': {
            'Supercell Size': 'p(4x2)' ,
            'Number of Layers': '5(2) O-Ti-O' ,
            'K-point Mesh': '2x2x1(1x1x1)',
            'Energy cutoff': '520',
            'Vacuum': '13',
            'Metal PAW potential': 'Ti (18e core)',
        }
    },
    'a-TiO2': {
        '(meta)GGA': {
            'Supercell Size': '3x1' ,
            'Number of Layers': '8(2) O-Ti-O layers',
            'K-point Mesh': '3x3x1',
            'Energy cutoff': '600',
            'Vacuum': '15',
            'Metal PAW potential': 'Ti_pv (12e core)',
        },
        'hybrid': {
            'Supercell Size': '3x1' ,
            'Number of Layers': '8(2) O-Ti-O' ,
            'K-point Mesh': '3x3x1(1x1x1)',
            'Energy cutoff': '520',
            'Vacuum': '13',
            'Metal PAW potential': 'Ti (18e core)',
        }
    }
}

dft_parameters_df = pd.DataFrame.from_dict(
    {(convert_to_nice_labels['surface'][surface], xc_func): parameters
     for surface, xc_func_dict in dft_parameters_dict.items()
     for xc_func, parameters in xc_func_dict.items()},
    orient='index'
).transpose()

# Write the DataFrame to a latex input
convert_df_to_latex_input(
    dft_parameters_df,
    start_input = r'\begin{table}',
    label = 'tab:dft_parameters',
    caption = 'DFT parameters used for the three different surfaces. The parameters for hybrids are also different from those for the metaGGA, GGA and vdW-inclusive functionals, grouped as (meta)GGA in the table. The number of layers in parentheses indicates the number of layers fixed at the bottom of the slab. The k-points in parenthesis indicates the k-point mesh used for the exact exchange potential',
    end_input = r'\end{table}',
    replace_input = {
        '_': r'\_',
        'x': r'${\times}$',
    },
    adjustbox = 1,
    df_latex_skip = 0,
    multiindex_sep = r'\cmidrule(lr){2-3}  \cmidrule(lr){4-5} \cmidrule(lr){6-7}',
    filename = 'Tables/SI_Table-DFT_Parameters.tex',
    column_format = 'l' + 'r'*len(dft_parameters_df.columns)
)

dft_parameters_df

MgO (001)                  \ce{TiO2} rutile (110)  \
                           (meta)GGA           hybrid              (meta)GGA   
Supercell Size                   4x4              4x4                 p(4x2)   
Number of Layers                4(2)             4(2)            5(3) O-Ti-O   
K-point Mesh                   2x2x1     2x2x1(2x2x1)                  2x2x1   
Energy cutoff                    600              600                    600   
Vacuum                            15               15                     15   
Metal PAW potential  Mg_pv (4e core)  Mg_pv (4e core)       Ti_pv (12e core)   

                                   \ce{TiO2} anatase (101)                 
                            hybrid               (meta)GGA         hybrid  
Supercell Size              p(4x2)                     3x1            3x1  
Number of Layers       5(2) O-Ti-O      8(2) O-Ti-O layers    8(2) O-Ti-O  
K-point Mesh          2x2x1(1x1x1)                   3x3x1   3x3x1(1x1x1)  
Energy cutoff                  520                     600            520  
Vacuum                          13                      15             13  
Metal PAW potential  Ti (18e core)        Ti_pv (12e core)  Ti (18e core)

### SI - DFT Lattice Parameters

In [83]:

# Initialize an empty list to store lattice parameters for each DFT functional
lattice_parameter_xc_dict = {xc_func: {'MgO a': 0, 'MgO Error': 0,  'r-TiO2 a': 0, 'r-TiO2 c': 0, 'r-TiO2 RMSE': 0, 'a-TiO2 a': 0, 'a-TiO2 c': 0, 'a-TiO2 RMSE': 0} for xc_func in ['Experiment'] + list(convert_to_nice_labels['xc_functionals'].values())}

lattice_parameter_xc_dict['Experiment'] = {'MgO a': 4.216, 'MgO Error': 0, 'r-TiO2 a': 4.587, 'r-TiO2 c': 2.954, 'r-TiO2 RMSE': 0, 'a-TiO2 a': 3.782, 'a-TiO2 c': 9.502, 'a-TiO2 RMSE': 0} 

# Loop through each DFT functional
for surface in ['MgO', 'r-TiO2', 'a-TiO2']:
    for xc_func in crystal_xc_func_ensemble[surface]:
        # Read the CONTCAR file to obtain lattice parameter 'a'
        if surface == 'r-TiO2' and xc_func == '06_HSE06-D4':
            continue
        unit_cell = io.read(f'Data/01-Unit_Cell/{surface}/{xc_func}/OUTCAR')
        # Add the lattice parameter 'a' to the latpar_functional list
        lattice_parameter_xc_dict[convert_to_nice_labels['xc_functionals'][xc_func]][f'{surface} a'] = unit_cell.get_cell()[0][0]
        if 'TiO2' in surface:
            lattice_parameter_xc_dict[convert_to_nice_labels['xc_functionals'][xc_func]][f'{surface} c'] = unit_cell.get_cell()[2][2]
            lattice_parameter_xc_dict[convert_to_nice_labels['xc_functionals'][xc_func]][f'{surface} RMSE'] = np.sqrt(np.sum((np.array([lattice_parameter_xc_dict['Experiment'][f'{surface} a'], lattice_parameter_xc_dict['Experiment'][f'{surface} c']]) - np.array([unit_cell.get_cell()[0][0], unit_cell.get_cell()[2][2]]))**2))
        else:
            lattice_parameter_xc_dict[convert_to_nice_labels['xc_functionals'][xc_func]][f'{surface} Error'] = np.abs(lattice_parameter_xc_dict['Experiment'][f'{surface} a'] - unit_cell.get_cell()[0][0])
        
# Create a DataFrame from the lattice_parameter_xc_dict
lattice_parameter_xc_df = pd.DataFrame(lattice_parameter_xc_dict).T
lattice_parameter_xc_df = lattice_parameter_xc_df.round(3)
lattice_parameter_xc_df.columns = ['$a$', 'Error', '$a$', '$c$', 'RMSE', '$a$', '$c$', 'RMSE']

# Write the DataFrame to a latex input
convert_df_to_latex_input(
    lattice_parameter_xc_df,
    start_input = r'\begin{table}',
    label = 'tab:lattice_parameters',
    caption = 'Lattice parameters for MgO, TiO$_2$ rutile, and TiO$_2$ anatase obtained from DFT calculations and experiment.',
    end_input = r'\end{table}',
    replace_input = {
        '&  0.000': '&      -'
    },
    adjustbox = 1,
    df_latex_skip = 0,
    filename = 'Tables/SI_Table-Lattice_Parameters.tex',
    column_format = 'l' + 'r'*len(lattice_parameter_xc_df.columns)
)

lattice_parameter_xc_df

,$a$,Error,$a$,$c$,RMSE,$a$,$c$,RMSE
Experiment,4.216,0.000,4.587,2.954,0.000,3.782,9.502,0.000
PBE-D2[Ne],4.234,0.018,0.000,0.000,0.000,0.000,0.000,0.000
revPBE-D4,4.220,0.004,4.598,2.958,0.011,3.790,9.549,0.048
vdW-DF,4.273,0.057,4.685,2.995,0.107,3.839,9.767,0.272
rev-vdW-DF2,4.220,0.004,4.618,2.961,0.032,3.798,9.593,0.093
PBE0-D4,4.175,0.041,0.000,0.000,0.000,0.000,0.000,0.000
B3LYP-D2[Ne],4.202,0.014,0.000,0.000,0.000,0.000,0.000,0.000
PBE+U-TS/HI,0.000,0.000,4.647,3.026,0.094,3.859,9.676,0.190
r$^2$SCAN-rVV10,0.000,0.000,4.590,2.957,0.004,3.786,9.531,0.029
HSE06-D4,0.000,0.000,0.000,0.000,0.000,3.769,9.518,0.020


### SI - *E*<sub>int</sub> for as function of (XC functional ensemble) geometry and single-point (XC functional ensemble)

In [84]:
dft_eint_dict = {surface : { molecule : {xc_func_sp : {xc_func_geom : 0 for xc_func_geom in crystal_xc_func_ensemble[surface]} for xc_func_sp in crystal_xc_func_ensemble[surface]} for molecule in molecule_surface_systems[surface]} for surface in ['MgO','r-TiO2','a-TiO2']}

for surface in ['MgO','r-TiO2','a-TiO2']:
    for molecule in molecule_surface_systems[surface]:
        molecule_label = molecule.replace(' ', '_')
        num_monomer = 4 if 'Tetramer' in molecule else 2 if 'Dimer' in molecule else 1
        for xc_func_sp in crystal_xc_func_ensemble[surface]: # the XC functional used for the geometry optimization
            for xc_func_geom in crystal_xc_func_ensemble[surface]: # the XC functional used for the single-point calculation
                dft_eint_dict[surface][molecule][xc_func_sp][xc_func_geom] = calculate_eint(f'Data/05a-Eint_and_Erlx_DFT/{surface}/{molecule_label}/{xc_func_sp}/{xc_func_geom}', code='vasp')*1000/num_monomer



### SI - *E*<sub>rlx</sub> for XC functional ensemble for monomers and monolayers

In [85]:
# Erlx is calculated with the XC functional used to originally generate the geometry

dft_eads_true_dict = {surface : { molecule : {xc_func : 0 for xc_func in crystal_xc_func_ensemble[surface]} for molecule in molecule_surface_systems[surface]} for surface in ['MgO','r-TiO2','a-TiO2']}
dft_erlx_dict = {surface : { molecule : {xc_func_sp : 0 for xc_func_sp in crystal_xc_func_ensemble[surface]} for molecule in molecule_surface_systems[surface]} for surface in ['MgO','r-TiO2','a-TiO2']}

for surface in ['MgO','r-TiO2','a-TiO2']:
    for molecule in dft_eads_true_dict[surface]:
        molecule_label = molecule.replace(' ', '_')
        num_monomer = 4 if 'Tetramer' in molecule else 4 if 'Monolayer' in molecule else 2 if 'Dimer' in molecule else 1
        for xc_func in crystal_xc_func_ensemble[surface]:
            molecule_surface_energy = get_energy(f'Data/04-Molecule_Surface/{surface}/{molecule_label}/{xc_func}/OUTCAR', code='vasp')
            if molecule in molecule_surface_systems['MgO_8x8']:
                molecule_surface_energy = get_energy(f'Data/05a-Eint_and_Erlx_DFT/{surface}/{molecule_label}/{xc_func}/{xc_func}/Molecule-Surface/OUTCAR', code='vasp')
                surface_energy = get_energy(f'Data/02-Surface/MgO_8x8/{xc_func}/OUTCAR', code='vasp')
                molecule_energy = get_energy(f'Data/03-Molecule/{surface}/{molecule_label}/{xc_func}/OUTCAR', code='vasp')
            else:
                surface_energy = get_energy(f'Data/02-Surface/{surface}/{xc_func}/OUTCAR', code='vasp')
                molecule_energy = get_energy(f'Data/03-Molecule/{surface}/{molecule.split()[0]}/{xc_func}/OUTCAR', code='vasp')

            dft_eads_true_dict[surface][molecule][xc_func] = (molecule_surface_energy - surface_energy - num_monomer*molecule_energy)*1000/num_monomer

            if molecule in ['CH4 Monolayer', 'C2H6 Monolayer', 'CH3OH Tetramer','H2O Tetramer']:
                dft_erlx_dict[surface][molecule][xc_func] = dft_eads_true_dict[surface][molecule][xc_func] - dft_eint_dict[surface][molecule][xc_func][xc_func]

            if molecule not in ['CH4 Monolayer', 'C2H6 Monolayer', 'CH3OH Tetramer','H2O Tetramer','NO Dimer','CO2 Chemisorbed']:
                dft_erlx_dict[surface][molecule][xc_func] = dft_eads_true_dict[surface][molecule][xc_func] - dft_eint_dict[surface][molecule][xc_func][xc_func]

### SI - *E*<sub>coh</sub>, *E*<sub>conf</sub> and *E*<sub>rlx</sub> for the monolayer, clusters and chemisorbed CO2 with the XC functional ensemble

In [86]:
# Calculating the conformational energy for the CO2 geometry chemisorbed to the MgO surface back to its pristine geometry

dft_econf = { molecule : {xc_func_sp : {xc_func_geom : 0 for xc_func_geom in crystal_xc_func_ensemble['MgO']} for xc_func_sp in crystal_xc_func_ensemble['MgO']} for molecule in ['CO2 Chemisorbed','NO Dimer']}

for molecule in ['CO2 Chemisorbed','NO Dimer']:
    molecule_label = molecule.replace(' ', '_')
    for xc_func_sp in crystal_xc_func_ensemble['MgO']:
        for xc_func_geom in crystal_xc_func_ensemble['MgO']:
            gas_phase_co2_energy = get_energy(f'Data/07-Econf/MgO/{molecule_label}/{xc_func_sp}/{xc_func_geom}/Molecule/OUTCAR', code='vasp')
            chemisorbed_co2_energy = get_energy(f'Data/07-Econf/MgO/{molecule_label}/{xc_func_sp}/{xc_func_geom}/Molecule-Surface/OUTCAR', code='vasp')
            dft_econf[molecule][xc_func_sp][xc_func_geom] = (chemisorbed_co2_energy - gas_phase_co2_energy)*1000

        if molecule == 'CO2 Chemisorbed':
            dft_erlx_dict['MgO']['CO2 Chemisorbed'][xc_func_sp] = dft_eads_true_dict['MgO']['CO2 Chemisorbed'][xc_func_sp] - dft_eint_dict['MgO']['CO2 Chemisorbed'][xc_func_sp][xc_func_sp] - dft_econf['CO2 Chemisorbed'][xc_func_sp][xc_func_sp]


In [87]:
# Calculating the cohesive energy that bind the molecules within the cluster or monolayer systems

dft_cluster_monolayer_ecoh = { molecule : {xc_func_sp : {xc_func_geom : 0 for xc_func_geom in crystal_xc_func_ensemble['MgO']} for xc_func_sp in crystal_xc_func_ensemble['MgO']} for molecule in ['CH4 Monolayer', 'C2H6 Monolayer', 'CH3OH Tetramer','H2O Tetramer','NO Dimer']}

for molecule in dft_cluster_monolayer_ecoh:
    if molecule == 'NO Dimer':
        continue

    molecule_label = molecule.replace(' ', '_')
    num_monomer = 4 if 'Tetramer' in molecule else 4 if 'Monolayer' in molecule else 2 if 'Dimer' in molecule else 1
    for xc_func_sp in crystal_xc_func_ensemble['MgO']:
        for xc_func_geom in crystal_xc_func_ensemble['MgO']:
            if 'Monolayer' in molecule:
                monolayer_energy = get_energy(f'Data/08-Ecoh/MgO/{molecule_label}/{xc_func_sp}/{xc_func_geom}/Monolayer/OUTCAR', code='vasp')
                monomer_energy = []
                for monomer_idx in range(1, num_monomer+1):
                    monomer_energy.append(get_energy(f'Data/08-Ecoh/MgO/{molecule_label}/{xc_func_sp}/{xc_func_geom}/Monolayer_Monomer_{monomer_idx}/OUTCAR', code='vasp'))
                dft_cluster_monolayer_ecoh[molecule][xc_func_sp][xc_func_geom] = (monolayer_energy/num_monomer - np.mean(monomer_energy))*1000

            else:
                cluster_energy = get_energy(f'Data/08-Ecoh/MgO/{molecule_label}/{xc_func_sp}/{xc_func_geom}/Cluster/OUTCAR', code='vasp')
                monomer_energy = []
                for monomer_idx in range(1, num_monomer+1):
                    monomer_energy.append(get_energy(f'Data/08-Ecoh/MgO/{molecule_label}/{xc_func_sp}/{xc_func_geom}/Cluster_Monomer_{monomer_idx}/OUTCAR', code='vasp'))
                dft_cluster_monolayer_ecoh[molecule][xc_func_sp][xc_func_geom] = (cluster_energy/num_monomer - np.mean(monomer_energy))*1000
        dft_erlx_dict['MgO'][molecule][xc_func_sp] = dft_eads_true_dict['MgO'][molecule][xc_func_sp] - dft_eint_dict['MgO'][molecule][xc_func_sp][xc_func_sp] - dft_cluster_monolayer_ecoh[molecule][xc_func_sp][xc_func_sp]


 ### SI - Comparing the true and approximate *E*<sub>ads</sub> value to estimate errors in the final *E*<sub>ads</sub>

In [88]:
dft_eads_approx_dict = {surface : { molecule : {xc_func_sp : {xc_func_geom : 0 for xc_func_geom in crystal_xc_func_ensemble[surface]} for xc_func_sp in crystal_xc_func_ensemble[surface]} for molecule in molecule_surface_systems[surface]} for surface in ['MgO','r-TiO2','a-TiO2']}

dft_erlx_error_dict = {surface : { molecule : {xc_func_sp : {xc_func_geom : 0 for xc_func_geom in crystal_xc_func_ensemble[surface]} for xc_func_sp in crystal_xc_func_ensemble[surface] + ['RMS']} for molecule in molecule_surface_systems[surface]} for surface in ['MgO','r-TiO2','a-TiO2']}

for surface in ['MgO','r-TiO2','a-TiO2']:
    for molecule in dft_erlx_error_dict[surface]:
        if molecule == 'NO Dimer':
            continue
        for xc_func_idx, xc_func_sp in enumerate(crystal_xc_func_ensemble[surface]):
            cumulative_squared_error = 0

            for xc_func_geom in crystal_xc_func_ensemble[surface]:
                if molecule in ['CH4 Monolayer', 'C2H6 Monolayer', 'CH3OH Tetramer','H2O Tetramer']:
                    dft_eads_approx_dict[surface][molecule][xc_func_sp][xc_func_geom] = dft_eint_dict[surface][molecule][xc_func_sp][xc_func_geom] + dft_erlx_dict[surface][molecule][xc_func_geom] + dft_cluster_monolayer_ecoh[molecule][xc_func_sp][xc_func_geom]
                elif molecule == 'CO2 Chemisorbed':
                    dft_eads_approx_dict[surface][molecule][xc_func_sp][xc_func_geom] = dft_eint_dict[surface][molecule][xc_func_sp][xc_func_geom] + dft_erlx_dict[surface][molecule][xc_func_geom] + dft_econf[molecule][xc_func_sp][xc_func_geom]
                else:
                    dft_eads_approx_dict[surface][molecule][xc_func_sp][xc_func_geom] = dft_eint_dict[surface][molecule][xc_func_sp][xc_func_geom] + dft_erlx_dict[surface][molecule][xc_func_geom]


                dft_erlx_error_dict[surface][molecule][xc_func_sp][xc_func_geom] = dft_eads_true_dict[surface][molecule][xc_func_sp] - dft_eads_approx_dict[surface][molecule][xc_func_sp][xc_func_geom]
                if xc_func_geom != xc_func_sp:
                    cumulative_squared_error += dft_erlx_error_dict[surface][molecule][xc_func_sp][xc_func_geom]**2

            dft_erlx_error_dict[surface][molecule][xc_func_sp]['RMS'] = np.sqrt(cumulative_squared_error/5)

### SI - Estimating Errors in *E*<sub>ads</sub> from the DFT XC ensemble

#### Monomers on MgO(001)

In [89]:
# Starting with the single monomers on MgO surface

dft_eads_geom_error_dict = {surface : { molecule : {xc_func: {'Eint': 0, 'Erlx': 0, 'Ecoh': 0, 'Econf': 0, 'Approx': 0, 'True': 0, 'Error': 0} for xc_func in crystal_xc_func_ensemble[surface]} for molecule in molecule_surface_systems[surface]} for surface in ['MgO','r-TiO2','a-TiO2']}

mgo_monomer_molecules = [molecule for molecule in molecule_surface_systems['MgO'] if ('Tetramer' not in molecule) and ('Dimer' not in molecule) and ('Monolayer' not in molecule)]

for surface in ['MgO']:
    for molecule in mgo_monomer_molecules:
        cumulative_squared_error = 0
        for xc_func in crystal_xc_func_ensemble[surface]:
            dft_eads_geom_error_dict[surface][molecule][xc_func]['Eint'] = dft_eint_dict[surface][molecule][xc_func]['02_revPBE-D4']
            dft_eads_geom_error_dict[surface][molecule][xc_func]['Erlx'] = dft_erlx_dict[surface][molecule]['02_revPBE-D4']
            dft_eads_geom_error_dict[surface][molecule][xc_func]['True'] = dft_eads_true_dict[surface][molecule][xc_func]
            dft_eads_geom_error_dict[surface][molecule][xc_func]['Approx'] = dft_eads_approx_dict[surface][molecule][xc_func]['02_revPBE-D4']
            dft_eads_geom_error_dict[surface][molecule][xc_func]['Error'] = dft_eads_geom_error_dict[surface][molecule][xc_func]['True'] - dft_eads_geom_error_dict[surface][molecule][xc_func]['Approx']
            cumulative_squared_error += dft_eads_geom_error_dict[surface][molecule][xc_func]['Error']**2
        dft_eads_geom_error_dict[surface][molecule]['RMSE'] = np.sqrt(cumulative_squared_error/5)
        


# Create a DataFrame from the dft_eads_geom_error_dict

dft_eads_geom_error_df_dict = {(f"{convert_to_nice_labels['xc_functionals'][xc_func]}//revPBE-D4" , quantity): [dft_eads_geom_error_dict['MgO'][molecule][xc_func][quantity] for molecule in mgo_monomer_molecules] for xc_func in crystal_xc_func_ensemble['MgO'] for quantity in ['Eint', 'Erlx', 'Approx', 'True', 'Error']}

dft_eads_geom_error_df_dict[(r'DeltaGeom', 'RMSE')] = [dft_eads_geom_error_dict['MgO'][molecule]['RMSE'] for molecule in mgo_monomer_molecules]


dft_eads_geom_error_df = pd.DataFrame.from_dict(dft_eads_geom_error_df_dict)


dft_eads_geom_error_df.index = [ molecule.split()[1] + r' \ce{'  + molecule.split()[0] + r'}' if len(molecule.split()) == 2 else r'\ce{' + molecule + r'}'   for molecule in mgo_monomer_molecules]

dft_eads_geom_error_df = dft_eads_geom_error_df.round().astype(int)

display(dft_eads_geom_error_df)

# Write the DataFrame to a latex input
convert_df_to_latex_input(
    dft_eads_geom_error_df,
    start_input = '\\begin{turnpage}\n\\begin{table}',
    label = 'tab:eads_dft_ensemble_errors',
    caption = r'For the monomers on the MgO(001) surface, we estimate the errors for using the revPBE-D4 geometry and $E_\textrm{rlx}$ in the final $E_\textrm{ads}$ of the autoSKZCAM protocol using an ensemble of 6 different DFT functionals. The errors are calculated as the difference between the true $E_\textrm{ads}^\textrm{true}$ (using the appropriate DFT functional) and the approximated $E_\textrm{ads}^\textrm{approx}$ using the revPBE-D4 geometry and $E_\textrm{rlx}$.',
    end_input = '\\end{table}\n\\end{turnpage}',
    replace_input = {
        'Erlx': r'$E_\textrm{rlx}$',
        'Eint': r'$E_\textrm{int}$',
        'True': r'$E_\textrm{ads}^\textrm{true}$',
        'Approx': r'$E_\textrm{ads}^\textrm{approx}$',
        'DeltaGeom': r'$\epsilon_\textrm{geom}$'

    },
    adjustbox = 1.1,
    center = True,
    df_latex_skip = 0,
    filename = 'Tables/SI_Table-Eads_Errors_Monomers_on_MgO.tex')


PBE-D2[Ne]//revPBE-D4                         \
                                         Eint Erlx Approx True Error   
\ce{CH4}                                 -115    2   -114 -115    -2   
\ce{C2H6}                                -157    1   -156 -158    -2   
\ce{CO}                                  -233    8   -224 -228    -3   
\ce{C6H6}                                -278   26   -252 -261    -9   
Parallel \ce{N2O}                        -180    3   -177 -179    -2   
Tilted \ce{N2O}                          -137   10   -127 -132    -5   
Vertical-Hollow \ce{NO}                  -263   29   -234 -184    50   
Vertical-Mg \ce{NO}                      -170    9   -161 -162    -1   
Bent-Bridge \ce{NO}                      -353   51   -302 -301     1   
Bent-Mg \ce{NO}                          -207    6   -201 -203    -2   
Bent-O \ce{NO}                           -242   39   -203 -196     7   
Monomer \ce{H2O}                         -666  114   -552 -561   -10   
Tilted \ce{CH3OH}                        -735  145   -590 -599    -9   
Parallel \ce{CH3OH}                      -460   46   -414 -417    -3   
\ce{NH3}                                 -617   89   -528 -540   -12   
Physisorbed \ce{CO2}                     -238   14   -225 -227    -2   
Chemisorbed \ce{CO2}                    -2993  663   -489 -518   -29   

                        revPBE-D4//revPBE-D4                         ...  \
                                        Eint Erlx Approx True Error  ...   
\ce{CH4}                                -143    2   -142 -142     0  ...   
\ce{C2H6}                               -219    1   -218 -218     0  ...   
\ce{CO}                                 -215    8   -206 -206     0  ...   
\ce{C6H6}                               -565   26   -538 -538     0  ...   
Parallel \ce{N2O}                       -192    3   -189 -189     0  ...   
Tilted \ce{N2O}                         -133   10   -123 -123     0  ...   
Vertical-Hollow \ce{NO}                 -184   29   -154 -154     0  ...   
Vertical-Mg \ce{NO}                     -142    9   -133 -133     0  ...   
Bent-Bridge \ce{NO}                     -346   51   -294 -294     0  ...   
Bent-Mg \ce{NO}                         -184    6   -178 -178     0  ...   
Bent-O \ce{NO}                          -234   39   -195 -195     0  ...   
Monomer \ce{H2O}                        -637  114   -522 -522     0  ...   
Tilted \ce{CH3OH}                       -770  145   -625 -625     0  ...   
Parallel \ce{CH3OH}                     -502   46   -457 -457     0  ...   
\ce{NH3}                                -636   89   -547 -547     0  ...   
Physisorbed \ce{CO2}                    -268   14   -254 -254     0  ...   
Chemisorbed \ce{CO2}                   -2981  663   -521 -521     0  ...   

                        PBE0-D4//revPBE-D4                    \
                                      Erlx Approx True Error   
\ce{CH4}                                 2   -158 -158     0   
\ce{C2H6}                                1   -226 -229    -4   
\ce{CO}                                  8   -241 -234     7   
\ce{C6H6}                               26   -497 -521   -24   
Parallel \ce{N2O}                        3   -243 -251    -8   
Tilted \ce{N2O}                         10   -156 -160    -4   
Vertical-Hollow \ce{NO}                 29    -74 -102   -28   
Vertical-Mg \ce{NO}                      9   -117 -123    -6   
Bent-Bridge \ce{NO}                     51   -208 -192    16   
Bent-Mg \ce{NO}                          6   -165 -172    -6   
Bent-O \ce{NO}                          39   -128 -123     5   
Monomer \ce{H2O}                       114   -626 -574    52   
Tilted \ce{CH3OH}                      145   -708 -659    49   
Parallel \ce{CH3OH}                     46   -491 -469    22   
\ce{NH3}                                89   -613 -576    37   
Physisorbed \ce{CO2}                    14   -312 -304     7   
Chemisorbed \ce{CO2}                   663   -898 -764   1

#### Clusters and Monolayer on MgO(001)

In [90]:
mgo_monolayer_cluster_molecules = [molecule for molecule in molecule_surface_systems['MgO'] if ('Tetramer' in molecule) or ('Monolayer' in molecule)]

for surface in ['MgO']:
    for molecule in mgo_monolayer_cluster_molecules:
        cumulative_squared_error = 0
        for xc_func in crystal_xc_func_ensemble[surface]:
            dft_eads_geom_error_dict[surface][molecule][xc_func]['Eint'] = dft_eint_dict[surface][molecule][xc_func]['02_revPBE-D4']
            dft_eads_geom_error_dict[surface][molecule][xc_func]['Erlx'] = dft_erlx_dict[surface][molecule]['02_revPBE-D4']
            dft_eads_geom_error_dict[surface][molecule][xc_func]['Ecoh'] = dft_cluster_monolayer_ecoh[molecule][xc_func]['02_revPBE-D4']
            dft_eads_geom_error_dict[surface][molecule][xc_func]['True'] = dft_eads_true_dict[surface][molecule][xc_func]
            dft_eads_geom_error_dict[surface][molecule][xc_func]['Approx'] = dft_eads_approx_dict[surface][molecule][xc_func]['02_revPBE-D4']
            dft_eads_geom_error_dict[surface][molecule][xc_func]['Error'] = dft_eads_geom_error_dict[surface][molecule][xc_func]['True'] - dft_eads_geom_error_dict[surface][molecule][xc_func]['Approx']
            cumulative_squared_error += dft_eads_geom_error_dict[surface][molecule][xc_func]['Error']**2
        dft_eads_geom_error_dict[surface][molecule]['RMSE'] = np.sqrt(cumulative_squared_error/5)
        


# Create a DataFrame from the dft_eads_geom_error_dict

dft_eads_geom_error_df_dict = {(f"{convert_to_nice_labels['xc_functionals'][xc_func]}//revPBE-D4" , quantity): [dft_eads_geom_error_dict['MgO'][molecule][xc_func][quantity] for molecule in mgo_monolayer_cluster_molecules] for xc_func in crystal_xc_func_ensemble['MgO'] for quantity in ['Eint', 'Erlx', 'Ecoh', 'Approx', 'True', 'Error']}

dft_eads_geom_error_df_dict[(r'DeltaGeom', 'RMSE')] = [dft_eads_geom_error_dict['MgO'][molecule]['RMSE'] for molecule in mgo_monolayer_cluster_molecules]


dft_eads_geom_error_df = pd.DataFrame.from_dict(dft_eads_geom_error_df_dict)


dft_eads_geom_error_df.index = [ molecule.split()[1] + r' \ce{'  + molecule.split()[0] + r'}' if len(molecule.split()) == 2 else r'\ce{' + molecule + r'}'   for molecule in mgo_monolayer_cluster_molecules]

dft_eads_geom_error_df = dft_eads_geom_error_df.round().astype(int)

display(dft_eads_geom_error_df)

# Write the DataFrame to a latex input
convert_df_to_latex_input(
    dft_eads_geom_error_df,
    start_input = '\\begin{turnpage}\n\\begin{table}',
    label = 'tab:eads_dft_ensemble_errors',
    caption = r'For the clusters and monolayer systems on the MgO(001) surface, we estimate the errors for using the revPBE-D4 geometry and $E_\textrm{rlx}$ in the final $E_\textrm{ads}$ of the autoSKZCAM protocol using an ensemble of 6 different DFT functionals. The errors are calculated as the difference between the true $E_\textrm{ads}^\textrm{true}$ (using the appropriate DFT functional) and the approximated $E_\textrm{ads}^\textrm{approx}$ using the revPBE-D4 geometry and $E_\textrm{rlx}$.',
    end_input = '\\end{table}\n\\end{turnpage}',
    replace_input = {
        'Erlx': r'$E_\textrm{rlx}$',
        'Eint': r'$E_\textrm{int}$',
        'Ecoh': r'$E_\textrm{coh}$',
        'True': r'$E_\textrm{ads}^\textrm{true}$',
        'Approx': r'$E_\textrm{ads}^\textrm{approx}$',
        'DeltaGeom': r'$\epsilon_\textrm{geom}$'

    },
    adjustbox = 1.1,
    center = True,
    df_latex_skip = 0,
    filename = 'Tables/SI_Table-Eads_Errors_Clusters_Monolayers_on_MgO.tex')


PBE-D2[Ne]//revPBE-D4                              \
                                     Eint Erlx Ecoh Approx True Error   
Monolayer \ce{CH4}                   -114   12  -53   -154 -153     1   
Monolayer \ce{C2H6}                  -142   16 -123   -250 -248     2   
Tetramer \ce{H2O}                    -397   52 -309   -654 -660    -6   
Tetramer \ce{CH3OH}                  -416   68 -380   -729 -733    -5   

                    revPBE-D4//revPBE-D4                   ...  \
                                    Eint Erlx Ecoh Approx  ...   
Monolayer \ce{CH4}                  -142   12  -42   -171  ...   
Monolayer \ce{C2H6}                 -207   16  -60   -251  ...   
Tetramer \ce{H2O}                   -404   52 -261   -613  ...   
Tetramer \ce{CH3OH}                 -483   68 -328   -743  ...   

                    PBE0-D4//revPBE-D4            B3LYP-D2[Ne]//revPBE-D4  \
                                Approx True Error                    Eint   
Monolayer \ce{CH4}                -190 -193    -3                     -87   
Monolayer \ce{C2H6}               -272 -274    -1                    -111   
Tetramer \ce{H2O}                 -715 -692    23                    -400   
Tetramer \ce{CH3OH}               -825 -795    30                    -429   

                                                DeltaGeom  
                    Erlx Ecoh Approx True Error      RMSE  
Monolayer \ce{CH4}    12  -40   -115 -127   -12        14  
Monolayer \ce{C2H6}   16 -124   -219 -238   -19        28  
Tetramer \ce{H2O}     52 -303   -651 -643     8        19  
Tetramer \ce{CH3OH}   68 -371   -732 -717    16        27  

[4 rows x 37 columns]

#### Monomers on TiO<sub>2</sub>

In [91]:
for surface in ['r-TiO2','a-TiO2']:
    for molecule in molecule_surface_systems[surface]:
        cumulative_squared_error = 0
        for xc_func in crystal_xc_func_ensemble[surface]:
            dft_eads_geom_error_dict[surface][molecule][xc_func]['Eint'] = dft_eint_dict[surface][molecule][xc_func]['02_revPBE-D4']
            dft_eads_geom_error_dict[surface][molecule][xc_func]['Erlx'] = dft_erlx_dict[surface][molecule]['02_revPBE-D4']
            dft_eads_geom_error_dict[surface][molecule][xc_func]['True'] = dft_eads_true_dict[surface][molecule][xc_func]
            dft_eads_geom_error_dict[surface][molecule][xc_func]['Approx'] = dft_eads_approx_dict[surface][molecule][xc_func]['02_revPBE-D4']
            dft_eads_geom_error_dict[surface][molecule][xc_func]['Error'] = dft_eads_geom_error_dict[surface][molecule][xc_func]['True'] - dft_eads_geom_error_dict[surface][molecule][xc_func]['Approx']
            cumulative_squared_error += dft_eads_geom_error_dict[surface][molecule][xc_func]['Error']**2
        dft_eads_geom_error_dict[surface][molecule]['RMSE'] = np.sqrt(cumulative_squared_error/5)
        
# Create a DataFrame from the dft_eads_geom_error_dict

dft_eads_geom_error_df_dict = {(f"{convert_to_nice_labels['xc_functionals'][xc_func]}//revPBE-D4" , quantity): [dft_eads_geom_error_dict[surface][molecule][xc_func][quantity] for surface in ['r-TiO2','a-TiO2'] for molecule in molecule_surface_systems[surface]] for xc_func in crystal_xc_func_ensemble['r-TiO2'] for quantity in ['Eint', 'Erlx', 'Approx', 'True', 'Error']}

dft_eads_geom_error_df_dict[(r'DeltaGeom', 'RMSE')] = [dft_eads_geom_error_dict[surface][molecule]['RMSE'] for surface in ['r-TiO2','a-TiO2'] for molecule in molecule_surface_systems[surface]]


dft_eads_geom_error_df = pd.DataFrame.from_dict(dft_eads_geom_error_df_dict)
dft_eads_geom_error_df.index = [ molecule.split()[1] + r' \ce{'  + molecule.split()[0] + r'} on ' + convert_to_nice_labels['surface'][surface] if len(molecule.split()) == 2 else r'\ce{' + molecule + r'} on ' + convert_to_nice_labels['surface'][surface] for surface in ['r-TiO2','a-TiO2'] for molecule in molecule_surface_systems[surface]]

dft_eads_geom_error_df = dft_eads_geom_error_df.round().astype(int).T

display(dft_eads_geom_error_df)

# Write the DataFrame to a latex input
convert_df_to_latex_input(
    dft_eads_geom_error_df,
    start_input = '\\begin{table}',
    label = 'tab:eads_dft_ensemble_errors',
    caption = r'For the monomers on the \ce{TiO2} rutile (110) and anatase (101) surfaces, we estimate the errors for using the revPBE-D4 geometry and $E_\textrm{rlx}$ in the final $E_\textrm{ads}$ of the autoSKZCAM protocol using an ensemble of 6 different DFT functionals. The errors are calculated as the difference between the true $E_\textrm{ads}^\textrm{true}$ (using the appropriate DFT functional) and the approximated $E_\textrm{ads}^\textrm{approx}$ using the revPBE-D4 geometry and $E_\textrm{rlx}$.',
    end_input = '\\end{table}\n',
    replace_input = {
        'Erlx': r'$E_\textrm{rlx}$',
        'Eint': r'$E_\textrm{int}$',
        'True': r'$E_\textrm{ads}^\textrm{true}$',
        'Approx': r'$E_\textrm{ads}^\textrm{approx}$',
        'DeltaGeom': r'$\epsilon_\textrm{geom}$'

    },
    adjustbox = 0.85,
    center = True,
    df_latex_skip = 0,
    filename = 'Tables/SI_Table-Eads_Errors_Monomers_on_TiO2.tex',
    column_format = 'll' + 'r'*len(dft_eads_geom_error_df.columns),
    rotate_column_header = True
    )


\ce{CH4} on \ce{TiO2} rutile (110)  \
PBE+U-TS/HI//revPBE-D4     Eint                                  -294   
                           Erlx                                    20   
                           Approx                                -274   
                           True                                  -302   
                           Error                                  -28   
revPBE-D4//revPBE-D4       Eint                                  -288   
                           Erlx                                    20   
                           Approx                                -268   
                           True                                  -268   
                           Error                                    0   
vdW-DF//revPBE-D4          Eint                                  -238   
                           Erlx                                    20   
                           Approx                                -218   
                           True                                  -244   
                           Error                                  -26   
rev-vdW-DF2//revPBE-D4     Eint                                  -274   
                           Erlx                                    20   
                           Approx                                -254   
                           True                                  -257   
                           Error                                   -3   
r$^2$SCAN-rVV10//revPBE-D4 Eint                                  -334   
                           Erlx                                    20   
                           Approx                                -314   
                           True                                  -325   
                           Error                                  -11   
HSE06-D4//revPBE-D4        Eint                                  -285   
                           Erlx                                    20   
                           Approx                                -265   
                           True                                  -271   
                           Error                                   -6   
DeltaGeom                  RMSE                                    18   

                                   Parallel \ce{CO2} on \ce{TiO2} rutile (110)  \
PBE+U-TS/HI//revPBE-D4     Eint                                           -345   
                           Erlx                                             14   
                           Approx                                         -331   
                           True                                           -396   
                           Error                                           -65   
revPBE-D4//revPBE-D4       Eint                                           -402   
                           Erlx                                             14   
                           Approx                                         -388   
                           True                                           -388   
                           Error                                             0   
vdW-DF//revPBE-D4          Eint                                           -368   
                           Erlx                                             14   
                           Approx                                         -354   
                           True                                           -380   
                           Error                                           -26   
rev-vdW-DF2//revPBE-D4     Eint                                           -409   
                           Erlx                                             14   
                           Approx                                         -395   
                           True                                           -401   
                           Error                          

### SI - Getting *E*<sub>ZPV</sub> and *E*<sub>therm</sub> with the DFT XC ensemble

In [92]:
dft_ezpv_etherm_dict = {surface : { molecule : {xc_func: {'E_ZPV': 0, 'E_therm': 0, 'kT': 0, 'DeltaH': 0} for xc_func in crystal_xc_func_ensemble[surface][:4] + ['Mean']} for molecule in molecule_surface_systems[surface]} for surface in ['MgO','r-TiO2','a-TiO2']}

dft_ezpv_etherm_df_dict = {} # DataFrame dictionary

for surface in ['MgO','r-TiO2','a-TiO2']:
    for molecule in molecule_surface_systems[surface]:
        if len(molecule.split()) == 2:
            system_label = convert_to_nice_labels['adsorbates'][surface][molecule] + f" on {convert_to_nice_labels['surface'][surface]}"
        else:
            system_label = convert_to_nice_labels['adsorbates'][surface][molecule] + f" on {convert_to_nice_labels['surface'][surface]}"

        if ('Tetramer' in molecule or 'NO' in molecule or 'CH3OH' in molecule) and surface == 'MgO':
            expt_molecule_label = 'Cluster ' + molecule.split()[0]
        elif 'Monolayer' in molecule:
            expt_molecule_label =  'Monolayer ' + molecule.split()[0]
        elif 'CO2' in molecule and surface == 'MgO':
            expt_molecule_label =  molecule.split()[1] + ' ' + molecule.split()[0]
        else:
            expt_molecule_label = molecule.split()[0]

        molecule_label = molecule.replace(' ', '_')
        num_monomer = 4 if 'Tetramer' in molecule else 4 if 'Monolayer' in molecule else 2 if 'Dimer' in molecule else 1
        temperature = float(expt_hads_df.loc[(expt_hads_df['Surface'] == surface) & (expt_hads_df['Molecule'] == expt_molecule_label), 'Temperature'].values[0])

        for xc_func in crystal_xc_func_ensemble[surface][:4]:
            dft_ezpv_etherm_dict[surface][molecule][xc_func]['E_ZPV'], dft_ezpv_etherm_dict[surface][molecule][xc_func]['E_therm'], dft_ezpv_etherm_dict[surface][molecule][xc_func]['kT'], dft_ezpv_etherm_dict[surface][molecule][xc_func]['DeltaH'] = calculate_ezpv_etherm(f'Data/06-Etherm_and_EZPV_DFT/{surface}/{molecule_label}/{xc_func}', ['Molecule-Surface','Molecule'], temperature, num_monomers=num_monomer)

        dft_ezpv_etherm_dict[surface][molecule]['Mean']['DeltaH'] = np.mean([dft_ezpv_etherm_dict[surface][molecule][xc_func]['DeltaH'] for xc_func in crystal_xc_func_ensemble[surface][:4]])
        dft_ezpv_etherm_dict[surface][molecule]['Mean']['Error'] = 2*np.std([dft_ezpv_etherm_dict[surface][molecule][xc_func]['DeltaH'] for xc_func in crystal_xc_func_ensemble[surface][:4]])

        dft_ezpv_etherm_df_dict[system_label] = [temperature, dft_ezpv_etherm_dict[surface][molecule][xc_func]['kT']] + [dft_ezpv_etherm_dict[surface][molecule][xc_func][quantity] for xc_func_idx, xc_func in enumerate(crystal_xc_func_ensemble[surface][:4]) for quantity in ['E_ZPV','E_therm','DeltaH']] +  [ dft_ezpv_etherm_dict[surface][molecule]['Mean']['DeltaH'], dft_ezpv_etherm_dict[surface][molecule]['Mean']['Error']]

# Create a DataFrame from the dft_ezpv_etherm_df_dict
dft_ezpv_etherm_df = pd.DataFrame.from_dict(dft_ezpv_etherm_df_dict)
dft_ezpv_etherm_df.index = ['Temperature [K]', 'kT [meV]'] + [f"{quantity} [XC {xc_func_idx+1}]" for xc_func_idx, xc_func in enumerate(crystal_xc_func_ensemble[surface][:4]) for quantity in ['E_ZPV','E_therm','DeltaH']] + ['Final DeltaH','Error']

# Convert to int
dft_ezpv_etherm_df = dft_ezpv_etherm_df.round().astype(int)

display(dft_ezpv_etherm_df)

# Write the DataFrame to a latex input
convert_df_to_latex_input(
    dft_ezpv_etherm_df,
    start_input = '\\begin{turnpage}\n\\begin{table}',
    label = 'tab:ethermal',
    caption = r'The zero-point vibrational energy ($E_\textrm{ZPV}$), thermal energy ($E_\textrm{therm}$), and overall enthalpy ($\Delta H$) contributions to the adsorption enthalpy for all studied systems using an ensemble of 4 different DFT functionals. The errors are calculated as the 2$\sigma$ standard deviation of the $\Delta H$ values for the 4 DFT functionals.',
    end_input = '\\end{table}\n\\end{turnpage}',
    adjustbox = 1.4,
    center = True,
    df_latex_skip = 0,
    filename = 'Tables/SI_Table-Ethermal.tex',
    replace_input = {
        'E_ZPV': r'$E_\textrm{ZPV}$',
        'E_therm': r'$E_\textrm{therm}$',
        'DeltaH': r'$\Delta H$'
    },
    rotate_column_header = True
)

,\ce{CH4} on MgO (001),Monolayer \ce{CH4} on MgO (001),\ce{C2H6} on MgO (001),Monolayer \ce{C2H6} on MgO (001),\ce{CO} on MgO (001),\ce{C6H6} on MgO (001),Parallel \ce{N2O} on MgO (001),Tilted \ce{N2O} on MgO (001),Vertical-Hollow \ce{NO} on MgO (001),Vertical-Mg \ce{NO} on MgO (001),...,\ce{NH3} on MgO (001),Physisorbed \ce{CO2} on MgO (001),Chemisorbed \ce{CO2} on MgO (001),\ce{CH4} on \ce{TiO2} rutile (110),Parallel \ce{CO2} on \ce{TiO2} rutile (110),Tilted \ce{CO2} on \ce{TiO2} rutile (110),\ce{H2O} on \ce{TiO2} rutile (110),\ce{CH3OH} on \ce{TiO2} rutile (110),\ce{H2O} on \ce{TiO2} anatase (101),\ce{NH3} on \ce{TiO2} anatase (101)
Temperature [K],47,47,75,75,61,162,77,77,80,80,...,160,120,230,85,177,177,303,370,257,410
kT [meV],4,4,6,6,5,14,7,7,7,7,...,14,10,20,7,15,15,26,32,22,35
E_ZPV [XC 1],29,29,12,17,31,1,1,3,18,13,...,72,2,47,22,13,13,102,58,113,111
E_therm [XC 1],-5,-5,-2,-4,-6,2,-1,0,-5,-3,...,-13,0,-8,-5,2,3,-23,7,-18,-2
DeltaH [XC 1],20,20,3,7,20,-12,-6,-4,7,3,...,46,-8,19,10,0,1,53,34,73,74
E_ZPV [XC 2],23,28,13,37,28,9,2,6,22,14,...,72,1,47,25,9,12,111,74,112,115
E_therm [XC 2],-4,-5,-2,-5,-5,3,-1,-1,-5,-4,...,-12,0,-8,-4,3,3,-15,9,-15,1
DeltaH [XC 2],15,20,4,26,18,-3,-5,-2,10,4,...,46,-9,19,13,-3,0,69,51,75,80
E_ZPV [XC 3],10,10,0,28,28,-1,0,0,12,10,...,69,-2,44,6,5,7,96,71,90,105
E_therm [XC 3],-2,-2,0,-5,-5,2,0,0,-3,-3,...,-12,1,-8,-1,3,3,-7,13,-9,3


#### Verifying accuracy of freezing surface in vibrational frequency calculations

In [93]:
dft_deltaH_surface_contribution_dict = {molecule: {'Molecule': '', 'Molecule+Surface': ''} for molecule in ['CO', 'H2O Monomer','CO2 Physisorbed','CO2 Chemisorbed']}

surface = 'MgO'

for molecule in ['CO', 'H2O Monomer','CO2 Physisorbed','CO2 Chemisorbed']:

    if 'CO2' in molecule and surface == 'MgO':
        expt_molecule_label =  molecule.split()[1] + ' ' + molecule.split()[0]
    else:
        expt_molecule_label = molecule.split()[0]

    molecule_label = molecule.replace(' ', '_')
    num_monomer = 1
    temperature = float(expt_hads_df.loc[(expt_hads_df['Surface'] == surface) & (expt_hads_df['Molecule'] == expt_molecule_label), 'Temperature'].values[0])

    deltaH_list = []
    for xc_func in crystal_xc_func_ensemble[surface][:4]:
        e_zpv, etherm, kT, deltaH = calculate_ezpv_etherm(f'Data/Miscellaneous/Etherm_and_EZPV_Surface_Test/{surface}/{molecule_label}/{xc_func}', ['Molecule-Surface','Molecule','Surface'], temperature, num_monomers=num_monomer)
        deltaH_list.append(deltaH)


    dft_deltaH_surface_contribution_dict[molecule]['Molecule'] = f'{round(dft_ezpv_etherm_dict[surface][molecule]["Mean"]["DeltaH"]):.0f} $\pm$ {round(2*dft_ezpv_etherm_dict[surface][molecule]["Mean"]["Error"]):.0f}'

    dft_deltaH_surface_contribution_dict[molecule]['Molecule+Surface'] = f'{round(np.mean(deltaH_list)):.0f} $\pm$ {round(2*np.std(deltaH_list)):.0f}'

# Convert to DataFrame then LaTeX input

dft_deltaH_surface_contribution_df = pd.DataFrame.from_dict(dft_deltaH_surface_contribution_dict)

display(dft_deltaH_surface_contribution_df)

# Write the DataFrame to a latex input
convert_df_to_latex_input(
    dft_deltaH_surface_contribution_df,
    start_input = '\\begin{table}',
    label = 'tab:ethermal_slab_effect',
    caption = r'Comparing the effect of only the molecule vibrational degrees of freedom and the inclusion of surface degrees of freedom on the enthalpy ($\Delta H$) contribution to the adsorption enthalpy for a select few molecules adsorbed on MgO (001). $\Delta H$ is calculated as the mean from an ensemble of 4 DFT XC functionals with the error being the 2$\sigma$.',
    end_input = '\\end{table}\n\\end{turnpage}',
    adjustbox = 1.4,
    center = True,
    df_latex_skip = 0,
    filename = 'Tables/SI_Table-Ethermal_Surface_Effect.tex',
    replace_input = {
        'E_ZPV': r'$E_\textrm{ZPV}$',
        'E_therm': r'$E_\textrm{therm}$',
        'DeltaH': r'$\Delta H$'
    },
    rotate_column_header = True
)

,CO,H2O Monomer,CO2 Physisorbed,CO2 Chemisorbed
Molecule,19 $\pm$ 6,47 $\pm$ 5,-10 $\pm$ 5,18 $\pm$ 5
Molecule+Surface,19 $\pm$ 1,48 $\pm$ 5,-8 $\pm$ 2,31 $\pm$ 3


# Computing *E*<sub>int</sub> with the SKZCAM protocol

### SI - *E*<sub>int</sub> as a function of SKZCAM cluster size/number

In [94]:
skzcam_cluster_size_dict = {surface: {molecule: {} for molecule in molecule_surface_systems[surface]} for surface in ['MgO','r-TiO2','a-TiO2']}
skzcam_cluster_eint_dict = {surface: {molecule: {} for molecule in molecule_surface_systems[surface]} for surface in ['MgO','r-TiO2','a-TiO2']}

for surface in ['MgO','r-TiO2','a-TiO2']:
    for molecule in molecule_surface_systems[surface]:
        molecule_label = molecule.replace(' ', '_')
        num_monomer = 4 if 'Tetramer' in molecule else 2 if 'Dimer' in molecule else 1
        for cluster_number in ['-2','-1','1','2','3','4','5','6','7','8']:
            # Looping over possibility of small metal cation ECP core and large metal cation ECP core
            for basis_type in ['aV','awCV']:
                filedir = f'Data/05b-Eint_SKZCAM/{surface}/{molecule_label}/Cluster_{cluster_number}'
                # First obtain the MP2 Eint values 
                outputname = f'MP2_{basis_type}'
                try:
                    # First get DZ Eint value
                    mp2_total_eint = calculate_skzcam_eint(filedir = filedir, outputname = outputname, code='orca', method='mp2_total', basis='DZ')/num_monomer
                    # If dictionary does not exist, create it
                    if cluster_number not in skzcam_cluster_eint_dict[surface][molecule]:
                        skzcam_cluster_eint_dict[surface][molecule][cluster_number] = {}
                        skzcam_cluster_size_dict[surface][molecule][cluster_number] = {}


                    skzcam_cluster_eint_dict[surface][molecule][cluster_number][f'MP2 {basis_type}DZ'] = mp2_total_eint
                    skzcam_cluster_size_dict[surface][molecule][cluster_number] = get_skzcam_cluster_size(f'{filedir}/Surface/{outputname}DZ.orca.out')
                except:
                    pass
                    
                try:
                    # Then get CBS(DZ/TZ) Eint value
                    hf_scf_eint = calculate_skzcam_eint(filedir = filedir, outputname = outputname, code='orca', method='mp2_hf', basis=['DZ','TZ'], cbs_type='scf_energy')/num_monomer
                    mp2_corr_eint = calculate_skzcam_eint(filedir = filedir, outputname = outputname, code='orca', method='mp2_corr', basis=['DZ','TZ'], cbs_type='correlation_energy')/num_monomer
                    skzcam_cluster_eint_dict[surface][molecule][cluster_number][f'MP2 CBS({basis_type}DZ/{basis_type}TZ)'] = hf_scf_eint + mp2_corr_eint
                except:
                    pass

                try:
                    # Then get CBS(TZ/QZ) Eint value
                    hf_scf_eint = calculate_skzcam_eint(filedir = filedir, outputname = outputname, code='orca', method='mp2_hf', basis=['TZ','QZ'], cbs_type='scf_energy')/num_monomer
                    mp2_corr_eint = calculate_skzcam_eint(filedir = filedir, outputname = outputname, code='orca', method='mp2_corr', basis=['TZ','QZ'], cbs_type='correlation_energy')/num_monomer
                    skzcam_cluster_eint_dict[surface][molecule][cluster_number][f'MP2 CBS({basis_type}TZ/{basis_type}QZ)'] = hf_scf_eint + mp2_corr_eint
                except:
                    pass

                # Then obtain the Eint values with CCSD(T)
                outputname = f'LNOCCSDT_{basis_type}'

                try:
                    # Get the CBS(DZ/TZ) LNO-CCSD(T) Eint value
                    hf_scf_eint = calculate_skzcam_eint(filedir = filedir, outputname = outputname, code='mrcc', method='hf', basis=['DZ','TZ'], cbs_type='scf_energy')/num_monomer
                    ccsdt_corr_eint = calculate_skzcam_eint(filedir = filedir, outputname = outputname, code='mrcc', method='lccsdt_corr', basis=['DZ','TZ'], cbs_type='correlation_energy')/num_monomer
                    ccsd_corr_eint = calculate_skzcam_eint(filedir = filedir, outputname = outputname, code='mrcc', method='lccsd_corr', basis=['DZ','TZ'], cbs_type='correlation_energy')/num_monomer
                    mp2_corr_eint = calculate_skzcam_eint(filedir = filedir, outputname = outputname, code='mrcc', method='lmp2_corr', basis=['DZ','TZ'], cbs_type='correlation_energy')/num_monomer
                    skzcam_cluster_eint_dict[surface][molecule][cluster_number][f'LNO-CCSD(T) CBS({basis_type}DZ/{basis_type}TZ)'] = hf_scf_eint + ccsdt_corr_eint
                    skzcam_cluster_eint_dict[surface][molecule][cluster_number][f'LNO-CCSD CBS({basis_type}DZ/{basis_type}TZ)'] = hf_scf_eint + ccsd_corr_eint
                    skzcam_cluster_eint_dict[surface][molecule][cluster_number][f'LMP2 CBS({basis_type}DZ/{basis_type}TZ)'] = hf_scf_eint + mp2_corr_eint
                except:
                    pass

                output_ccsdt_name = f'DLPNOCCSDT_{basis_type}'
                output_mp2_name = f'DLPNOMP2_{basis_type}'

                try:
                    # Get the CBS(DZ/TZ) LNO-CCSD(T) Eint value
                    hf_scf_eint = calculate_skzcam_eint(filedir = filedir, outputname = output_ccsdt_name, code='orca', method='dlpnoccsdt_hf', basis=['DZ','TZ'], cbs_type='scf_energy')/num_monomer
                    ccsdt_corr_eint = calculate_skzcam_eint(filedir = filedir, outputname = output_ccsdt_name, code='orca', method='dlpnoccsdt_corr', basis=['DZ','TZ'], cbs_type='correlation_energy')/num_monomer
                    ccsd_corr_eint = calculate_skzcam_eint(filedir = filedir, outputname = output_ccsdt_name, code='orca', method='dlpnoccsd_corr', basis=['DZ','TZ'], cbs_type='correlation_energy')/num_monomer
                    skzcam_cluster_eint_dict[surface][molecule][cluster_number][f'DLPNO-CCSD(T) CBS({basis_type}DZ/{basis_type}TZ)'] = hf_scf_eint + ccsdt_corr_eint
                    skzcam_cluster_eint_dict[surface][molecule][cluster_number][f'DLPNO-CCSD CBS({basis_type}DZ/{basis_type}TZ)'] = hf_scf_eint + ccsd_corr_eint


                    hf_scf_eint = calculate_skzcam_eint(filedir = filedir, outputname = output_mp2_name, code='orca', method='dlpnomp2_hf', basis=['DZ','TZ'], cbs_type='scf_energy')/num_monomer
                    mp2_corr_eint = calculate_skzcam_eint(filedir = filedir, outputname = output_mp2_name, code='orca', method='dlpnomp2_corr', basis=['DZ','TZ'], cbs_type='correlation_energy')/num_monomer
                    skzcam_cluster_eint_dict[surface][molecule][cluster_number][f'DLPNO-MP2 CBS({basis_type}DZ/{basis_type}TZ)'] = hf_scf_eint + mp2_corr_eint
                    # print(molecule,hf_scf_eint, mp2_corr_eint)
                except:
                    pass

                outputname = f'CCSDT_{basis_type}'

                try:
                    # Get the CBS(DZ/TZ) Canonical CCSD(T) Eint value
                    hf_scf_eint = calculate_skzcam_eint(filedir = filedir, outputname = outputname, code='mrcc', method='hf', basis=['DZ','TZ'], cbs_type='scf_energy')/num_monomer
                    ccsdt_corr_eint = calculate_skzcam_eint(filedir = filedir, outputname = outputname, code='mrcc', method='ccsdt_corr', basis=['DZ','TZ'], cbs_type='correlation_energy')/num_monomer
                    skzcam_cluster_eint_dict[surface][molecule][cluster_number][f'CCSD(T) CBS({basis_type}DZ/{basis_type}TZ)'] = hf_scf_eint + ccsdt_corr_eint

                except:
                    pass

### SI - Extrapolate *E*<sub>int</sub> to the bulk limit

In [95]:
# Now extrapolating the Eint values to the bulk limit

skzcam_extrap_eint_dict = {surface: {molecule: {} for molecule in molecule_surface_systems[surface]} for surface in ['MgO','r-TiO2','a-TiO2']}

latex_input_str = ""

for surface in ['MgO','r-TiO2','a-TiO2']:
    for molecule in molecule_surface_systems[surface]:
        molecule_label = molecule.replace(' ', '_')
        num_monomer = 4 if 'Tetramer' in molecule else 2 if 'Dimer' in molecule else 1
        basis_type = 'aV' if 'TiO2' in surface or ('NO' in molecule and 'NO Dimer' not in molecule) else 'awCV'
        system_skzcam_all_eint_dict = {} # A dictionary for writing into DataFrame
        system_skzcam_all_eint_dict[r'# of atoms'] = [skzcam_cluster_size_dict[surface][molecule][cluster_number] for cluster_number in ['1','2','3','4','5','6','7']]
        for basis in [f'{basis_type}DZ', f'CBS({basis_type}DZ/{basis_type}TZ)']:
            basis_eint_list = []
            basis_extrap_eint_list = []
            cluster_size_list = []

            for cluster_number in ['1','2','3','4','5','6','7']:
                try:
                    basis_eint_list += [skzcam_cluster_eint_dict[surface][molecule][cluster_number][f'MP2 {basis}']]
                    cluster_size_list += [skzcam_cluster_size_dict[surface][molecule][cluster_number]]



                    if int(cluster_number) > 3:
                        if cluster_number not in skzcam_extrap_eint_dict[surface][molecule]:
                            skzcam_extrap_eint_dict[surface][molecule][cluster_number] = {}

                        A_opt, gamma, eint_bulklim = fit_eint(cluster_size_list, basis_eint_list)
                        skzcam_extrap_eint_dict[surface][molecule][cluster_number][f'MP2 {basis}'] = eint_bulklim
                        basis_extrap_eint_list += [eint_bulklim]
                    else:
                        basis_extrap_eint_list += ['-']

                except:
                    basis_eint_list += ['-']
                    basis_extrap_eint_list += ['-']
                    pass
            system_skzcam_all_eint_dict[f'Eint MP2 {basis}'] = basis_eint_list
            system_skzcam_all_eint_dict[f'Eintbulk MP2 {basis}'] = basis_extrap_eint_list

        if basis_type == 'aV':
            extra_method_list = [f'MP2 CBS(awCVDZ/awCVTZ)', f'MP2 CBS({basis_type}TZ/{basis_type}QZ)', f'MP2 CBS(awCVTZ/awCVQZ)', f'LMP2 CBS({basis_type}DZ/{basis_type}TZ)',f'LNO-CCSD(T) CBS({basis_type}DZ/{basis_type}TZ)', f'DLPNO-MP2 CBS({basis_type}DZ/{basis_type}TZ)',f'DLPNO-CCSD(T) CBS({basis_type}DZ/{basis_type}TZ)' ]
        else:
            extra_method_list = [f'MP2 CBS({basis_type}TZ/{basis_type}QZ)', f'LMP2 CBS({basis_type}DZ/{basis_type}TZ)',f'LNO-CCSD(T) CBS({basis_type}DZ/{basis_type}TZ)']

        for method in extra_method_list:
            if method in skzcam_cluster_eint_dict[surface][molecule]['1']:
                system_skzcam_all_eint_dict[f'Eint {method}'] = [skzcam_cluster_eint_dict[surface][molecule][cluster_number][method] if method in skzcam_cluster_eint_dict[surface][molecule][cluster_number] else '-' for cluster_number in ['1', '2', '3', '4', '5', '6', '7']]
        system_skzcam_all_eint_df = pd.DataFrame.from_dict(system_skzcam_all_eint_dict).T
        system_skzcam_all_eint_df.columns = list(range(1,len(system_skzcam_all_eint_df.columns)+1))
        # Function to round only floats
        system_skzcam_all_eint_df = system_skzcam_all_eint_df.map(lambda x: int(round(x)) if isinstance(x, float) else x)

        # Identify the type of SKZCAM clusters in Figure X of the SI
        SKZCAM_cluster_type = 'none'

        if system_skzcam_all_eint_dict[r'# of atoms'][0] == 6:
            SKZCAM_cluster_type = 'Mg-centered MgO (001)'
        elif system_skzcam_all_eint_dict[r'# of atoms'][0] == 17:
            SKZCAM_cluster_type = 'O-centered MgO (001)'
        elif system_skzcam_all_eint_dict[r'# of atoms'][0] == 10:
            SKZCAM_cluster_type = 'Hollow-centered MgO (001)'
        elif system_skzcam_all_eint_dict[r'# of atoms'][0] == 34:
            SKZCAM_cluster_type = r'\ce{TiO2} rutile (110)'
        elif system_skzcam_all_eint_dict[r'# of atoms'][0] == 29:
            SKZCAM_cluster_type = r'\ce{TiO2} anatase (101)'

        # if molecule in adsorbate_config[system]:
        caption = """$E_\\textrm{int}$ (in meV) of the SKZCAM clusters for """ + f"{convert_to_nice_labels['adsorbates'][surface][molecule]} on {convert_to_nice_labels['surface'][surface]}." + f" The clusters correspond to the `{SKZCAM_cluster_type}' type visualized in Figure" + r"~\ref{fig:skzcam_clusters}."

        system_skzcam_all_eint_df.index.name = 'Cluster'

        latex_input_str += '\n' + convert_df_to_latex_input(
    system_skzcam_all_eint_df,
    start_input = '\\begin{table}',
    label = f'tab:system_eint_{surface.lower()}_{molecule_label.lower()}',
    caption = caption,
    end_input = '\\end{table}',
    adjustbox = 1.4,
    center = True,
    df_latex_skip = 0,
    filename = 'Tables/SI_Table-SKZCAM_System_Eint.tex',
    replace_input = {
        'Eintbulk': r'$E_\textrm{int}^\textrm{bulk}$',
        'Eint': r'$E_\textrm{int}$',
        r'#': r'\#'
    },
    output_str = True
    ) + '\n'
        
# Write out the latex input string
with open('Tables/SI_Table-SKZCAM_System_Eint.tex', 'w') as f:
    f.write(latex_input_str)


### SI - Validating local approximations in ΔCC correction

In [96]:
# Table comparing the LNO-CCSD(T) estimate against the CCSD(T) estimate for all of the systems

deltacc_methods = ['C-MP2','L-MP2','L-MP2 Error','C-CCSD(T)','LNO-CCSD(T)','LNO-CCSD(T) Error','C-DeltaCC','(L-)DeltaCC','(L-)DeltaCC Error']

deltacc_local_error_dict = {} # {f'{convert_to_nice_labels["adsorbates"][surface][molecule]} on {convert_to_nice_labels["surface"][surface]}': {method: 0 for method in deltacc_methods} for surface in ['MgO','r-TiO2','a-TiO2'] for molecule in molecule_surface_systems[surface]}

deltacc_method_mad_dict = {method: [] for method in deltacc_methods}


for surface in ['MgO','r-TiO2','a-TiO2']:
    cluster_number = '1' if surface == 'MgO' else '-2'
    basis_type = 'aV' if 'TiO2' in surface else 'awCV'
    if surface == 'MgO':
        molecule_list = ['CH4', 'C2H6', 'CO', 'N2O Parallel', 'N2O Tilted', 'NO Dimer', 'H2O Monomer', 'CH3OH Tilted', 'CH3OH Parallel', 'NH3', 'CO2 Physisorbed']
    else:
        molecule_list = molecule_surface_systems[surface]
    for molecule in molecule_list:
        lmp2_type = 'DLPNO-MP2' if (molecule == 'NO' and molecule != 'NO Dimer')  else 'LMP2'
        lccsdt_type = 'DLPNO-CCSD(T)' if (molecule == 'NO' and molecule != 'NO Dimer') else 'LNO-CCSD(T)'
        deltacc_local_error_dict[f'{convert_to_nice_labels["adsorbates"][surface][molecule]} on {convert_to_nice_labels["surface"][surface]}'] = {
                'C-MP2': skzcam_cluster_eint_dict[surface][molecule][cluster_number][f'MP2 CBS({basis_type}DZ/{basis_type}TZ)'],
                'L-MP2': skzcam_cluster_eint_dict[surface][molecule][cluster_number][f'{lmp2_type} CBS({basis_type}DZ/{basis_type}TZ)'],
                'L-MP2 Error': abs(skzcam_cluster_eint_dict[surface][molecule][cluster_number][f'MP2 CBS({basis_type}DZ/{basis_type}TZ)'] - skzcam_cluster_eint_dict[surface][molecule][cluster_number][f'{lmp2_type} CBS({basis_type}DZ/{basis_type}TZ)']),
                'C-CCSD(T)': skzcam_cluster_eint_dict[surface][molecule][cluster_number][f'CCSD(T) CBS({basis_type}DZ/{basis_type}TZ)'],
                'LNO-CCSD(T)': skzcam_cluster_eint_dict[surface][molecule][cluster_number][f'{lccsdt_type} CBS({basis_type}DZ/{basis_type}TZ)'],
                'LNO-CCSD(T) Error': abs(skzcam_cluster_eint_dict[surface][molecule][cluster_number][f'CCSD(T) CBS({basis_type}DZ/{basis_type}TZ)'] - skzcam_cluster_eint_dict[surface][molecule][cluster_number][f'{lccsdt_type} CBS({basis_type}DZ/{basis_type}TZ)']),
                'C-DeltaCC': skzcam_cluster_eint_dict[surface][molecule][cluster_number][f'CCSD(T) CBS({basis_type}DZ/{basis_type}TZ)'] - skzcam_cluster_eint_dict[surface][molecule][cluster_number][f'MP2 CBS({basis_type}DZ/{basis_type}TZ)'],
                '(L-)DeltaCC': skzcam_cluster_eint_dict[surface][molecule][cluster_number][f'{lccsdt_type} CBS({basis_type}DZ/{basis_type}TZ)'] - skzcam_cluster_eint_dict[surface][molecule][cluster_number][f'{lmp2_type} CBS({basis_type}DZ/{basis_type}TZ)'],
                '(L-)DeltaCC Error': abs((skzcam_cluster_eint_dict[surface][molecule][cluster_number][f'{lccsdt_type} CBS({basis_type}DZ/{basis_type}TZ)'] - skzcam_cluster_eint_dict[surface][molecule][cluster_number][f'{lmp2_type} CBS({basis_type}DZ/{basis_type}TZ)']) - (skzcam_cluster_eint_dict[surface][molecule][cluster_number][f'CCSD(T) CBS({basis_type}DZ/{basis_type}TZ)'] - skzcam_cluster_eint_dict[surface][molecule][cluster_number][f'MP2 CBS({basis_type}DZ/{basis_type}TZ)']))
            }
        for method in deltacc_methods:
            if 'Error' in method:
                deltacc_method_mad_dict[method] += [deltacc_local_error_dict[f'{convert_to_nice_labels["adsorbates"][surface][molecule]} on {convert_to_nice_labels["surface"][surface]}'][method]]
            else:
                deltacc_method_mad_dict[method] += [0]


deltacc_local_error_df = pd.DataFrame.from_dict(deltacc_local_error_dict).T

# Add a row for the mean absolute deviation
deltacc_local_error_df.loc['MAD'] = [np.mean(deltacc_method_mad_dict[method]) for method in deltacc_methods]

# Convert to int
deltacc_local_error_df = deltacc_local_error_df.round().astype(int)
display(deltacc_local_error_df)

# Write the DataFrame to a latex input
convert_df_to_latex_input(
    deltacc_local_error_df,
    start_input = '\\begin{table}',
    label = f'tab:deltacc_lno_errors',
    caption = r"Comparing canonical (C-)MP2, canonical CCSD(T) and canonical $\Delta$CC against their local variants (i.e., LMP2, LNO-CCSD(T), L-$\Delta$CC) used in the autoSKZCAM. This was compared for the first cluster generated by the SKZCAM protocol for MgO in Fig.~\ref{fig:skzcam_clusters}, while it corresponds to cluster ${-}2$ for the \ce{TiO2} surfaces.",
    end_input = '\\end{table}',
    adjustbox = 1,
    center = True,
    df_latex_skip = 0,
    rotate_column_header = True,
    filename = 'Tables/SI_Table-DeltaCC_LNO_Errors.tex')


,C-MP2,L-MP2,L-MP2 Error,C-CCSD(T),LNO-CCSD(T),LNO-CCSD(T) Error,C-DeltaCC,(L-)DeltaCC,(L-)DeltaCC Error
\ce{CH4} on MgO (001),-71,-70,0,-91,-86,5,-20,-15,4
\ce{C2H6} on MgO (001),-90,-92,2,-116,-112,4,-26,-20,6
\ce{CO} on MgO (001),-155,-153,2,-165,-160,5,-9,-7,3
Parallel \ce{N2O} on MgO (001),-206,-203,2,-215,-209,6,-9,-6,3
Tilted \ce{N2O} on MgO (001),-122,-120,2,-107,-104,4,15,17,2
Dimer \ce{NO} on MgO (001),-213,-208,5,-189,-183,7,24,26,2
Monomer \ce{H2O} on MgO (001),-641,-639,2,-670,-665,5,-29,-26,3
Tilted \ce{CH3OH} on MgO (001),-703,-701,2,-729,-723,7,-27,-22,5
Parallel \ce{CH3OH} on MgO (001),-443,-442,1,-466,-458,8,-23,-16,6
\ce{NH3} on MgO (001),-551,-547,4,-583,-578,5,-32,-31,1


### SI - ΔCC, Δ<sub>basis</sub> and Δ<sub>core</sub> corrections

In [97]:
skzcam_delta_dict = {surface: {molecule: {delta: np.array([0.0,0.0]) for delta in ['DeltaCC [CCSD(T)]', 'DeltaCC [CCSD]', 'DeltaBasis','DeltaCore']} for molecule in molecule_surface_systems[surface]} for surface in ['MgO','r-TiO2','a-TiO2']}

skzcam_delta_df_dict = {delta: {f'{convert_to_nice_labels["adsorbates"][surface][molecule]} on {convert_to_nice_labels["surface"][surface]}': [] for surface in ['MgO','r-TiO2','a-TiO2'] for molecule in molecule_surface_systems[surface]} for delta in ['DeltaCC [CCSD(T)]', 'DeltaBasis','DeltaCore']}

for surface in ['MgO','r-TiO2','a-TiO2']:
    for molecule in molecule_surface_systems[surface]:
        molecule_label = molecule.replace(' ', '_')
        basis_type = 'aV' if 'TiO2' in surface or ('NO' in molecule and 'NO Dimer' not in molecule) else 'awCV'

        deltacc_ccsdt_list = []
        deltacc_ccsd_list = []
        deltabasis_list = []
        deltacore_list = []
        for cluster_number in ['1','2','3','4','5','6','7']:
            # First obtain the deltaCC values                    
            try:
                if f'LNO-CCSD(T) CBS({basis_type}DZ/{basis_type}TZ)' in skzcam_cluster_eint_dict[surface][molecule][cluster_number]:
                    deltacc_ccsdt_list += [skzcam_cluster_eint_dict[surface][molecule][cluster_number][f'LNO-CCSD(T) CBS({basis_type}DZ/{basis_type}TZ)'] - skzcam_cluster_eint_dict[surface][molecule][cluster_number][f'LMP2 CBS({basis_type}DZ/{basis_type}TZ)']]
                    deltacc_ccsd_list += [skzcam_cluster_eint_dict[surface][molecule][cluster_number][f'LNO-CCSD CBS({basis_type}DZ/{basis_type}TZ)'] - skzcam_cluster_eint_dict[surface][molecule][cluster_number][f'LMP2 CBS({basis_type}DZ/{basis_type}TZ)']]
                elif f'DLPNO-CCSD(T) CBS({basis_type}DZ/{basis_type}TZ)' in skzcam_cluster_eint_dict[surface][molecule][cluster_number].keys():
                    deltacc_ccsdt_list += [skzcam_cluster_eint_dict[surface][molecule][cluster_number][f'DLPNO-CCSD(T) CBS({basis_type}DZ/{basis_type}TZ)'] - skzcam_cluster_eint_dict[surface][molecule][cluster_number][f'DLPNO-MP2 CBS({basis_type}DZ/{basis_type}TZ)']]
                    deltacc_ccsd_list += [skzcam_cluster_eint_dict[surface][molecule][cluster_number][f'DLPNO-CCSD CBS({basis_type}DZ/{basis_type}TZ)'] - skzcam_cluster_eint_dict[surface][molecule][cluster_number][f'DLPNO-MP2 CBS({basis_type}DZ/{basis_type}TZ)']]
            except:
                pass      

            # Next obtain the deltaBasis values

            try:
                if f'MP2 CBS({basis_type}TZ/{basis_type}QZ)' in skzcam_cluster_eint_dict[surface][molecule][cluster_number]:
                    deltabasis_list += [skzcam_cluster_eint_dict[surface][molecule][cluster_number][f'MP2 CBS({basis_type}TZ/{basis_type}QZ)'] - skzcam_cluster_eint_dict[surface][molecule][cluster_number][f'MP2 CBS({basis_type}DZ/{basis_type}TZ)']]
            except:
                pass

            # Finally obtain the deltaCore values

            if basis_type == 'aV':
                try:
                    if f'MP2 CBS(awCVTZ/awCVQZ)' in skzcam_cluster_eint_dict[surface][molecule][cluster_number]:
                        deltacore_list += [skzcam_cluster_eint_dict[surface][molecule][cluster_number][f'MP2 CBS(awCVTZ/awCVQZ)'] - skzcam_cluster_eint_dict[surface][molecule][cluster_number][f'MP2 CBS(aVTZ/aVQZ)']]
                    elif f'MP2 CBS(awCVDZ/awCVTZ)' in skzcam_cluster_eint_dict[surface][molecule][cluster_number]:
                        deltacore_list += [skzcam_cluster_eint_dict[surface][molecule][cluster_number][f'MP2 CBS(awCVDZ/awCVTZ)'] - skzcam_cluster_eint_dict[surface][molecule][cluster_number][f'MP2 CBS(aVDZ/aVTZ)']]
                except:
                    pass
            
        # Take DeltaCC to be the average of the list and its error to be the maximum deviation from the average
        skzcam_delta_dict[surface][molecule]['DeltaCC [CCSD(T)]'][0] = np.mean(deltacc_ccsdt_list)
        skzcam_delta_dict[surface][molecule]['DeltaCC [CCSD(T)]'][1] = np.max(np.abs(np.array(deltacc_ccsdt_list) - skzcam_delta_dict[surface][molecule]['DeltaCC [CCSD(T)]'][0]))   
        skzcam_delta_dict[surface][molecule]['DeltaCC [CCSD]'][0] = np.mean(deltacc_ccsd_list)
        skzcam_delta_dict[surface][molecule]['DeltaCC [CCSD]'][1] = np.max(np.abs(np.array(deltacc_ccsd_list) - skzcam_delta_dict[surface][molecule]['DeltaCC [CCSD]'][0]))   

        # Take DeltaBasis to be the average of the list and its error to be the maximum deviation from the average
        skzcam_delta_dict[surface][molecule]['DeltaBasis'][0] = np.mean(deltabasis_list)
        skzcam_delta_dict[surface][molecule]['DeltaBasis'][1] = np.max(np.abs(np.array(deltabasis_list) - skzcam_delta_dict[surface][molecule]['DeltaBasis'][0]))

        # Take DeltaCore to be the average of the list and its error to be the maximum deviation from the average
        if len(deltacore_list) > 1:
            skzcam_delta_dict[surface][molecule]['DeltaCore'][0] = np.mean(deltacore_list)
            skzcam_delta_dict[surface][molecule]['DeltaCore'][1] = np.max(np.abs(np.array(deltacore_list) - skzcam_delta_dict[surface][molecule]['DeltaCore'][0]))



        skzcam_delta_df_dict['DeltaCC [CCSD(T)]'][f'{convert_to_nice_labels["adsorbates"][surface][molecule]} on {convert_to_nice_labels["surface"][surface]}'] = deltacc_ccsdt_list + ['-']*(7-len(deltacc_ccsdt_list))
        skzcam_delta_df_dict['DeltaBasis'][f'{convert_to_nice_labels["adsorbates"][surface][molecule]} on {convert_to_nice_labels["surface"][surface]}'] = deltabasis_list + ['-']*(7-len(deltabasis_list))
        skzcam_delta_df_dict['DeltaCore'][f'{convert_to_nice_labels["adsorbates"][surface][molecule]} on {convert_to_nice_labels["surface"][surface]}'] = deltacore_list + ['-']*(7-len(deltacore_list))

delta_label_dict ={
    'DeltaCC [CCSD(T)]': r'$\Delta$CC',
    'DeltaBasis': r'$\Delta_\textrm{Basis}$',
    'DeltaCore': r'$\Delta_\textrm{Core}$'
}

for delta in ['DeltaCC [CCSD(T)]','DeltaBasis','DeltaCore']:
    
    delta_df = pd.DataFrame.from_dict(skzcam_delta_df_dict[delta]).T
    # Only keep first 3 columns
    delta_df = delta_df.iloc[:, :3]
    delta_df.columns = list(range(1,4))

 # for molecule in molecule_surface_systems['r-TiO2'] if molecule in delta_df.index]]

    # Add a column for the mean and another column for the maximum deviation
    delta_df['Mean'] = [skzcam_delta_dict[surface][molecule][delta][0] for surface in ['MgO','r-TiO2','a-TiO2'] for molecule in molecule_surface_systems[surface]]
    delta_df['Error'] = [skzcam_delta_dict[surface][molecule][delta][1] for surface in ['MgO','r-TiO2','a-TiO2'] for molecule in molecule_surface_systems[surface]]
    # Convert to int
    delta_df = delta_df.applymap(lambda x: int(round(x)) if isinstance(x, float) else x)
    # If DeltaCore, only keep rows with TiO2 or NO
    if delta == 'DeltaCore':
        delta_df = delta_df[(delta_df.index.str.contains('TiO2') | delta_df.index.str.contains('NO')) & ~delta_df.index.str.contains('Dimer')]
    delta_df.index.name = 'System'
    delta_df.columns.name = 'Cluster'
    delta_df.style.set_caption(delta)
    display(delta_df)
    # Convert to LaTeX input
    convert_df_to_latex_input(
        delta_df,
        start_input = '\\begin{table}',
        label = f'tab:{delta.split()[0].lower()}',
        caption = f'Comparison of the {delta_label_dict[delta]} values from the SKZCAM clusters for the studied systems. The mean is calculated from the set of clusters used, with the error being the maximum deviation from the mean.',
        end_input = '\\end{table}',
        adjustbox = 1,
        center = True,
        df_latex_skip = 0,
        filename = f'Tables/SI_Table-{delta.split()[0]}.tex',
)


/tmp/ipykernel_720/4054395148.py:85: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  delta_df = delta_df.applymap(lambda x: int(round(x)) if isinstance(x, float) else x)


Cluster,1,2,3,Mean,Error
System,,,,,
\ce{CH4} on MgO (001),-15,-16,-16,-16,0
Monolayer \ce{CH4} on MgO (001),-15,-16,-16,-16,0
\ce{C2H6} on MgO (001),-20,-20,-20,-20,0
Monolayer \ce{C2H6} on MgO (001),-19,-19,-20,-20,1
\ce{CO} on MgO (001),-7,-7,-9,-8,1
\ce{C6H6} on MgO (001),24,24,25,24,1
Parallel \ce{N2O} on MgO (001),-6,-7,-7,-6,1
Tilted \ce{N2O} on MgO (001),17,16,16,16,1
Vertical-Hollow \ce{NO} on MgO (001),289,-,-,289,0


/tmp/ipykernel_720/4054395148.py:85: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  delta_df = delta_df.applymap(lambda x: int(round(x)) if isinstance(x, float) else x)


Cluster,1,2,3,Mean,Error
System,,,,,
\ce{CH4} on MgO (001),3,0,2,2,1
Monolayer \ce{CH4} on MgO (001),3,1,2,2,1
\ce{C2H6} on MgO (001),3,3,4,3,1
Monolayer \ce{C2H6} on MgO (001),2,2,4,3,1
\ce{CO} on MgO (001),0,-1,3,1,2
\ce{C6H6} on MgO (001),-15,-15,-2,-11,9
Parallel \ce{N2O} on MgO (001),-2,-2,2,-1,2
Tilted \ce{N2O} on MgO (001),-6,-6,-3,-5,2
Vertical-Hollow \ce{NO} on MgO (001),10,8,8,9,1


/tmp/ipykernel_720/4054395148.py:85: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  delta_df = delta_df.applymap(lambda x: int(round(x)) if isinstance(x, float) else x)


Cluster,1,2,3,Mean,Error
System,,,,,
Vertical-Hollow \ce{NO} on MgO (001),-18,-24,-24,-22,4
Vertical-Mg \ce{NO} on MgO (001),-22,-26,-28,-25,3
Bent-Bridge \ce{NO} on MgO (001),-19,-26,-28,-24,5
Bent-Mg \ce{NO} on MgO (001),-22,-27,-29,-26,3
Bent-O \ce{NO} on MgO (001),-9,-15,-18,-14,5
\ce{CH4} on \ce{TiO2} rutile (110),-2,-3,-,-2,0
Parallel \ce{CO2} on \ce{TiO2} rutile (110),-8,-7,-,-7,0
Tilted \ce{CO2} on \ce{TiO2} rutile (110),8,10,-,9,1
\ce{H2O} on \ce{TiO2} rutile (110),37,44,-,41,4


### SI - Final *E*<sub>int</sub> estimate

In [98]:
skzcam_final_eint_dict = {surface: {molecule: {quantity: [0.0,0.0] for quantity in ['Final MP2', 'Final CCSD','Final CCSD(T)', 'Bulk MP2 Eint', 'DeltaCC [CCSD(T)]', 'DeltaCC [CCSD]', 'DeltaBasis','DeltaCore']} for molecule in molecule_surface_systems[surface]} for surface in ['MgO','r-TiO2','a-TiO2']}

for surface in ['MgO','r-TiO2','a-TiO2']:
    for molecule in molecule_surface_systems[surface]:
        basis_type = 'aV' if 'TiO2' in surface or ('NO' in molecule and 'NO Dimer' not in molecule) else 'awCV'
        cbs_cluster_number = [cluster_number for cluster_number in skzcam_cluster_eint_dict[surface][molecule].keys() if f'MP2 CBS({basis_type}DZ/{basis_type}TZ)' in skzcam_cluster_eint_dict[surface][molecule][cluster_number].keys()][-1]
        bulk_mp2_eint = skzcam_cluster_eint_dict[surface][molecule][cbs_cluster_number][f'MP2 CBS({basis_type}DZ/{basis_type}TZ)']
        bulk_mp2_eint_error = abs(skzcam_extrap_eint_dict[surface][molecule][cbs_cluster_number][f'MP2 {basis_type}DZ'] - skzcam_extrap_eint_dict[surface][molecule][list(skzcam_cluster_eint_dict[surface][molecule].keys())[-1]][f'MP2 {basis_type}DZ'])
        skzcam_final_eint_dict[surface][molecule]['Bulk MP2 Eint'] = [bulk_mp2_eint, bulk_mp2_eint_error]

        for quantity in ['DeltaCC [CCSD(T)]', 'DeltaCC [CCSD]', 'DeltaBasis','DeltaCore']:
            skzcam_final_eint_dict[surface][molecule][quantity] = skzcam_delta_dict[surface][molecule][quantity]

        # Now obtain the final Eint value for MP2
        skzcam_final_eint_dict[surface][molecule]['Final MP2'][0] = np.sum([skzcam_final_eint_dict[surface][molecule][quantity][0] for quantity in ['Bulk MP2 Eint', 'DeltaBasis','DeltaCore']])
        skzcam_final_eint_dict[surface][molecule]['Final MP2'][1] = np.sqrt(np.sum([skzcam_final_eint_dict[surface][molecule][quantity][1]**2 for quantity in ['Bulk MP2 Eint', 'DeltaBasis','DeltaCore']]))

        # Now obtain the final Eint value for CCSD
        skzcam_final_eint_dict[surface][molecule]['Final CCSD'][0] = np.sum([skzcam_final_eint_dict[surface][molecule][quantity][0] for quantity in ['Bulk MP2 Eint', 'DeltaCC [CCSD]', 'DeltaBasis','DeltaCore']])
        skzcam_final_eint_dict[surface][molecule]['Final CCSD'][1] = np.sqrt(np.sum([skzcam_final_eint_dict[surface][molecule][quantity][1]**2 for quantity in ['Bulk MP2 Eint', 'DeltaCC [CCSD]', 'DeltaBasis','DeltaCore']]))

        # Now obtain the final Eint value for CCSD(T)
        skzcam_final_eint_dict[surface][molecule]['Final CCSD(T)'][0] = np.sum([skzcam_final_eint_dict[surface][molecule][quantity][0] for quantity in ['Bulk MP2 Eint', 'DeltaCC [CCSD(T)]', 'DeltaBasis','DeltaCore']])
        skzcam_final_eint_dict[surface][molecule]['Final CCSD(T)'][1] = np.sqrt(np.sum([skzcam_final_eint_dict[surface][molecule][quantity][1]**2 for quantity in ['Bulk MP2 Eint', 'DeltaCC [CCSD(T)]', 'DeltaBasis','DeltaCore']]))

final_eint_df_dict = {f'{convert_to_nice_labels["adsorbates"][surface][molecule]} on {convert_to_nice_labels["surface"][surface]}': {quantity: f'{int(round(skzcam_final_eint_dict[surface][molecule][quantity][0]))} ± {int(round(skzcam_final_eint_dict[surface][molecule][quantity][1]))}' for quantity in ['Bulk MP2 Eint', 'DeltaCC [CCSD(T)]', 'DeltaCC [CCSD]', 'DeltaBasis','DeltaCore','Final MP2', 'Final CCSD','Final CCSD(T)']} for surface in ['MgO','r-TiO2','a-TiO2'] for molecule in molecule_surface_systems[surface]}

# Convert to DataFrame
final_eint_df = pd.DataFrame.from_dict(final_eint_df_dict).T

display(final_eint_df)
final_eint_df.index.name = 'System'
final_eint_df.columns.name = 'Quantity'
final_eint_df.columns = [r'$E_\textrm{int}^\textrm{bulk MP2}$', r'$\Delta$CC [CCSD(T)]', r'$\Delta$CC [CCSD]', r'$\Delta_\textrm{Basis}$', r'$\Delta_\textrm{Core}$', r'$E_\textrm{int}^\textrm{autoSKZCAM}$ [MP2]', r'$E_\textrm{int}^\textrm{autoSKZCAM}$ [CCSD]', r'$E_\textrm{int}^\textrm{autoSKZCAM}$ [CCSD(T)]']

# Write the DataFrame to a latex input
convert_df_to_latex_input(
    final_eint_df,
    start_input = '\\begin{table}',
    label = f'tab:final_eint',
    caption = r"Final E$_\textrm{int}$ values for the systems studied in this work. We show the individual terms which make up these final $E_\textrm{int}$ values and also give Final MP2, CCSD and CCSD(T) estimates, where the latter two are obtained by adding the $\Delta$CC values to the final MP2 E$_\textrm{int}$ value.",
    end_input = '\\end{table}',
    adjustbox = 0.9,
    center = True,
    df_latex_skip = 0,
    replace_input = {
        '±': r'$\pm$'
    },
    rotate_column_header = True,
    filename = 'Tables/SI_Table-Final_Eint.tex')


,Bulk MP2 Eint,DeltaCC [CCSD(T)],DeltaCC [CCSD],DeltaBasis,DeltaCore,Final MP2,Final CCSD,Final CCSD(T)
\ce{CH4} on MgO (001),-105 ± 2,-16 ± 0,18 ± 7,2 ± 1,0 ± 0,-104 ± 2,-86 ± 7,-120 ± 2
Monolayer \ce{CH4} on MgO (001),-104 ± 2,-16 ± 0,18 ± 7,2 ± 1,0 ± 0,-103 ± 3,-85 ± 7,-119 ± 3
\ce{C2H6} on MgO (001),-153 ± 3,-20 ± 0,32 ± 13,3 ± 1,0 ± 0,-150 ± 4,-118 ± 14,-170 ± 4
Monolayer \ce{C2H6} on MgO (001),-139 ± 3,-20 ± 1,29 ± 13,3 ± 1,0 ± 0,-136 ± 3,-108 ± 13,-156 ± 3
\ce{CO} on MgO (001),-197 ± 3,-8 ± 1,33 ± 8,1 ± 2,0 ± 0,-196 ± 3,-163 ± 9,-204 ± 4
\ce{C6H6} on MgO (001),-428 ± 2,24 ± 1,164 ± 24,-11 ± 9,0 ± 0,-439 ± 9,-275 ± 26,-415 ± 9
Parallel \ce{N2O} on MgO (001),-242 ± 2,-6 ± 1,40 ± 6,-1 ± 2,0 ± 0,-242 ± 3,-203 ± 7,-249 ± 3
Tilted \ce{N2O} on MgO (001),-175 ± 3,16 ± 1,64 ± 11,-5 ± 2,0 ± 0,-180 ± 4,-116 ± 11,-164 ± 4
Vertical-Hollow \ce{NO} on MgO (001),-241 ± 1,289 ± 0,334 ± 0,9 ± 1,-22 ± 4,-255 ± 4,79 ± 4,35 ± 4
Vertical-Mg \ce{NO} on MgO (001),30 ± 2,-68 ± 0,-49 ± 0,5 ± 4,-25 ± 3,10 ± 5,-39 ± 5,-58 ± 5


# Final *E*<sub>ads</sub> and *H*<sub>ads</sub> estimates from autoSKZCAM

### Getting the ΔCC for *E*<sub>coh</sub> and *E*<sub>conf</sub> corrections

In [99]:
wft_econf = { molecule : {xc_func_sp : {xc_func_geom : 0 for xc_func_geom in crystal_xc_func_ensemble['MgO']} for xc_func_sp in ['CCSD(T)','CCSD','MP2']} for molecule in ['CO2 Chemisorbed','NO Dimer']}

wft_cluster_monolayer_ecoh = { molecule : {xc_func_sp : {xc_func_geom : 0 for xc_func_geom in ['02_revPBE-D4']} for xc_func_sp in ['CCSD(T)','CCSD','MP2']} for molecule in ['CH4 Monolayer', 'C2H6 Monolayer', 'CH3OH Tetramer','H2O Tetramer']}

# Calculate wave-function theory Ecoh values for the clusters
for molecule in ['CH3OH Tetramer','H2O Tetramer']:
    molecule_label = molecule.replace(' ', '_')
    filedir = f'Data/08-Ecoh/MgO/{molecule_label}/07_CCSDT/02_revPBE-D4'
    hf_scf_energy = calculate_skzcam_eint(filedir = filedir, outputname = 'FNOCCSDT_aV', code='mrcc', method='hf', basis=['TZ','QZ'], cbs_type='scf_energy', structure_labels=['Cluster','Cluster_Monomer_1','Cluster_Monomer_2','Cluster_Monomer_3','Cluster_Monomer_4'])
    mp2_corr_energy = calculate_skzcam_eint(filedir = filedir, outputname = 'FNOCCSDT_aV', code='mrcc', method='fnomp2_corr', basis=['TZ','QZ'], cbs_type='correlation_energy', structure_labels=['Cluster','Cluster_Monomer_1','Cluster_Monomer_2','Cluster_Monomer_3','Cluster_Monomer_4'])
    ccsd_corr_energy = calculate_skzcam_eint(filedir = filedir, outputname = 'FNOCCSDT_aV', code='mrcc', method='fnoccsd_corr', basis=['TZ','QZ'], cbs_type='correlation_energy', structure_labels=['Cluster','Cluster_Monomer_1','Cluster_Monomer_2','Cluster_Monomer_3','Cluster_Monomer_4'])
    ccsdt_corr_energy = calculate_skzcam_eint(filedir = filedir, outputname = 'FNOCCSDT_aV', code='mrcc', method='fnoccsdt_corr', basis=['TZ','QZ'], cbs_type='correlation_energy', structure_labels=['Cluster','Cluster_Monomer_1','Cluster_Monomer_2','Cluster_Monomer_3','Cluster_Monomer_4'])
    wft_cluster_monolayer_ecoh[molecule]['CCSD(T)']['02_revPBE-D4'] = (hf_scf_energy + ccsdt_corr_energy)/4 
    wft_cluster_monolayer_ecoh[molecule]['CCSD']['02_revPBE-D4'] = (hf_scf_energy + ccsd_corr_energy)/4
    wft_cluster_monolayer_ecoh[molecule]['MP2']['02_revPBE-D4'] = (hf_scf_energy + mp2_corr_energy)/4  

# Now get the Ecoh values for the monolayers using a two-body many-body expansion
for molecule in ['CH4 Monolayer', 'C2H6 Monolayer']:
    two_body_terms = 68 if molecule == 'CH4 Monolayer' else 72 if molecule == 'C2H6 Monolayer' else 0
    molecule_label = molecule.replace(' ', '_')
    hf_scf_energy = 0
    mp2_corr_energy = 0
    ccsd_corr_energy = 0
    ccsdt_corr_energy = 0
    revpbe_d4_energy = 0
    for dimer_idx in range(1,two_body_terms+1):
        filedir = f'Data/08-Ecoh/MgO/{molecule_label}/07_CCSDT_2B/02_revPBE-D4/{dimer_idx:02d}'
        hf_scf_energy += calculate_skzcam_eint(filedir = filedir, outputname = f'FNOCCSDT_aV', code='mrcc', method='hf', basis=['TZ','QZ'], cbs_type='scf_energy', structure_labels=['Dimer','Monomer_A','Monomer_B'])/2
        mp2_corr_energy += calculate_skzcam_eint(filedir = filedir, outputname = f'FNOCCSDT_aV', code='mrcc', method='fnomp2_corr', basis=['TZ','QZ'], cbs_type='correlation_energy', structure_labels=['Dimer','Monomer_A','Monomer_B'])/2
        ccsd_corr_energy += calculate_skzcam_eint(filedir = filedir, outputname = f'FNOCCSDT_aV', code='mrcc', method='fnoccsd_corr', basis=['TZ','QZ'], cbs_type='correlation_energy', structure_labels=['Dimer','Monomer_A','Monomer_B'])/2
        ccsdt_corr_energy += calculate_skzcam_eint(filedir = filedir, outputname = f'FNOCCSDT_aV', code='mrcc', method='fnoccsdt_corr', basis=['TZ','QZ'], cbs_type='correlation_energy', structure_labels=['Dimer','Monomer_A','Monomer_B'])/2
        filedir = f'Data/08-Ecoh/MgO/{molecule_label}/02_revPBE-D4_2B/02_revPBE-D4/{dimer_idx:02d}'

        revpbe_d4_energy += calculate_skzcam_eint(filedir = filedir, outputname = f'revPBE-D4_QZVPD', code='orca', method='dft_total', basis='', cbs_type='scf_energy', structure_labels=['Dimer','Monomer_A','Monomer_B'])/2


    wft_cluster_monolayer_ecoh[molecule]['CCSD(T)']['02_revPBE-D4'] = hf_scf_energy + ccsdt_corr_energy - revpbe_d4_energy + dft_cluster_monolayer_ecoh[molecule]['02_revPBE-D4']['02_revPBE-D4']
    wft_cluster_monolayer_ecoh[molecule]['CCSD']['02_revPBE-D4'] = hf_scf_energy + ccsd_corr_energy - revpbe_d4_energy + dft_cluster_monolayer_ecoh[molecule]['02_revPBE-D4']['02_revPBE-D4']
    wft_cluster_monolayer_ecoh[molecule]['MP2']['02_revPBE-D4'] = hf_scf_energy + mp2_corr_energy - revpbe_d4_energy + dft_cluster_monolayer_ecoh[molecule]['02_revPBE-D4']['02_revPBE-D4']

# Now get the Econf values for chemisorbed CO2 and NO dimer
for molecule in ['CO2 Chemisorbed','NO Dimer']:
    molecule_label = molecule.replace(' ', '_')
    num_monomer = 2 if 'NO Dimer' in molecule else 1
    for xc_func_geom in ['02_revPBE-D4','06_B3LYP-D2-Ne']:
        filedir = f'Data/07-Econf/MgO/{molecule_label}/07_CCSDT/{xc_func_geom}'
        hf_scf_energy = calculate_skzcam_eint(filedir = filedir, outputname = f'CCSDT_aV', code='mrcc', method='hf', basis=['TZ','QZ'], cbs_type='scf_energy', structure_labels=['Molecule-Surface','Molecule'])/num_monomer
        mp2_corr_energy = calculate_skzcam_eint(filedir = filedir, outputname = f'CCSDT_aV', code='mrcc', method='mp2_corr', basis=['TZ','QZ'], cbs_type='correlation_energy', structure_labels=['Molecule-Surface','Molecule'])/num_monomer
        ccsd_corr_energy = calculate_skzcam_eint(filedir = filedir, outputname = f'CCSDT_aV', code='mrcc', method='ccsd_corr', basis=['TZ','QZ'], cbs_type='correlation_energy', structure_labels=['Molecule-Surface','Molecule'])/num_monomer
        ccsdt_corr_energy = calculate_skzcam_eint(filedir = filedir, outputname = f'CCSDT_aV', code='mrcc', method='ccsdt_corr', basis=['TZ','QZ'], cbs_type='correlation_energy', structure_labels=['Molecule-Surface','Molecule'])/num_monomer
        wft_econf[molecule]['CCSD(T)'][xc_func_geom] = hf_scf_energy + ccsdt_corr_energy
        wft_econf[molecule]['CCSD'][xc_func_geom] = hf_scf_energy + ccsd_corr_energy
        wft_econf[molecule]['MP2'][xc_func_geom] = hf_scf_energy + mp2_corr_energy

### Final *E*<sub>ads</sub> estimates

In [100]:
autoskzcam_final_eads_dict = {surface: {molecule: {quantity: [0.0,0.0] for quantity in ['SKZCAM Eint','DFT Erlx', 'CCSD(T) Ecoh', 'CCSD(T) Econf', 'Final']} for molecule in molecule_surface_systems[surface]} for surface in ['MgO','r-TiO2','a-TiO2']}

for surface in ['MgO','r-TiO2','a-TiO2']:
    for molecule in molecule_surface_systems[surface]:
        molecule_label = molecule.replace(' ', '_')
        if molecule == 'NO Dimer':
            continue

        # First obtain the SKZCAM Eint values
        autoskzcam_final_eads_dict[surface][molecule]['SKZCAM Eint'] = [skzcam_final_eint_dict[surface][molecule]['Final CCSD(T)'][0], skzcam_final_eint_dict[surface][molecule]['Final CCSD(T)'][1]]

        # Then obtain the DFT Erlx values
        autoskzcam_final_eads_dict[surface][molecule]['DFT Erlx'] = [dft_erlx_dict[surface][molecule]['02_revPBE-D4'], dft_erlx_error_dict[surface][molecule]['02_revPBE-D4']['RMS']]

        # Then obtain the CCSD(T) Ecoh values
        if molecule in wft_cluster_monolayer_ecoh:
            autoskzcam_final_eads_dict[surface][molecule]['CCSD(T) Ecoh'] = [wft_cluster_monolayer_ecoh[molecule]['CCSD(T)']['02_revPBE-D4'], 0]

        # Then obtain the CCSD(T) Econf values
        if molecule in wft_econf:
            autoskzcam_final_eads_dict[surface][molecule]['CCSD(T) Econf'] = [wft_econf[molecule]['CCSD(T)']['02_revPBE-D4'], 0]

        autoskzcam_final_eads_dict[surface][molecule]['Final'] = [np.sum([autoskzcam_final_eads_dict[surface][molecule][quantity][0] for quantity in ['SKZCAM Eint','DFT Erlx', 'CCSD(T) Ecoh', 'CCSD(T) Econf']]), np.sqrt(np.sum([autoskzcam_final_eads_dict[surface][molecule][quantity][1]**2 for quantity in ['SKZCAM Eint','DFT Erlx', 'CCSD(T) Ecoh', 'CCSD(T) Econf']]))]


### Final *H*<sub>ads</sub> estimates

In [111]:
autoskzcam_final_hads_dict = {surface: {molecule: {quantity: [0.0,0.0] for quantity in ['autoSKZCAM Eads', 'DFT DeltaH', 'Final']} for molecule in molecule_surface_systems[surface]} for surface in ['MgO','r-TiO2','a-TiO2']}

for surface in ['MgO','r-TiO2','a-TiO2']:
    for molecule in molecule_surface_systems[surface]:
        molecule_label = molecule.replace(' ', '_')
        if molecule == 'NO Dimer':
            continue

        # First obtain the SKZCAM Eint values
        autoskzcam_final_hads_dict[surface][molecule]['autoSKZCAM Eads'] = [autoskzcam_final_eads_dict[surface][molecule]['Final'][0], autoskzcam_final_eads_dict[surface][molecule]['Final'][1]]

        # Then obtain the DFT DeltaH values
        autoskzcam_final_hads_dict[surface][molecule]['DFT DeltaH'] = [dft_ezpv_etherm_dict[surface][molecule]['Mean']['DeltaH'], dft_ezpv_etherm_dict[surface][molecule]['Mean']['Error']]

        autoskzcam_final_hads_dict[surface][molecule]['Final'] = [np.sum([autoskzcam_final_hads_dict[surface][molecule][quantity][0] for quantity in ['autoSKZCAM Eads', 'DFT DeltaH']]), np.sqrt(np.sum([autoskzcam_final_hads_dict[surface][molecule][quantity][1]**2 for quantity in ['autoSKZCAM Eads', 'DFT DeltaH']]))]


# autoskzcam_final_hads_df_dict = {f'{convert_to_nice_labels["adsorbates"][surface][molecule]} on {convert_to_nice_labels["surface"][surface]}': {quantity: f'{int(round(autoskzcam_final_hads_dict[surface][molecule][quantity][0]))} ± {int(round(autoskzcam_final_hads_dict[surface][molecule][quantity][1]))}' for quantity in ['autoSKZCAM Eads', 'DFT DeltaH', 'Final']} for surface in ['MgO','r-TiO2','a-TiO2'] for molecule in molecule_surface_systems[surface]}

# autoskzcam_final_eads_df_dict = {f'{convert_to_nice_labels["adsorbates"][surface][molecule]} on {convert_to_nice_labels["surface"][surface]}': {quantity: f'{int(round(autoskzcam_final_eads_dict[surface][molecule][quantity][0]))} ± {int(round(autoskzcam_final_eads_dict[surface][molecule][quantity][1]))}' for quantity in ['SKZCAM Eint','DFT Erlx', 'CCSD(T) Ecoh', 'CCSD(T) Econf', 'Final']} for surface in ['MgO','r-TiO2','a-TiO2'] for molecule in molecule_surface_systems[surface]}


autoskzcam_hads_terms_df_dict = {
    f'{convert_to_nice_labels["adsorbates"][surface][molecule]} on {convert_to_nice_labels["surface"][surface]}': 
    {**{
        quantity: f'{int(round(autoskzcam_final_eads_dict[surface][molecule][quantity][0]))} ± {int(round(autoskzcam_final_eads_dict[surface][molecule][quantity][1]))}' 
        for quantity in ['SKZCAM Eint', 'DFT Erlx', 'CCSD(T) Ecoh', 'CCSD(T) Econf']
    },
    **{
        quantity: f'{int(round(autoskzcam_final_eads_dict[surface][molecule][quantity][0]))}' 
        for quantity in ['DFT Erlx', 'CCSD(T) Ecoh', 'CCSD(T) Econf']
    },  
    **{
        'epsilon geom': f'{int(round(autoskzcam_final_eads_dict[surface][molecule][quantity][1]))}' 
        for quantity in ['DFT Erlx']
    },      
    **{
        quantity: f'{int(round(autoskzcam_final_hads_dict[surface][molecule][quantity][0]))} ± {int(round(autoskzcam_final_hads_dict[surface][molecule][quantity][1]))}' 
        for quantity in ['autoSKZCAM Eads', 'DFT DeltaH', 'Final']
    }}
    for surface in ['MgO', 'r-TiO2', 'a-TiO2'] 
    for molecule in molecule_surface_systems[surface]
}

autoskzcam_hads_terms_df = pd.DataFrame.from_dict(autoskzcam_hads_terms_df_dict).T
autoskzcam_hads_terms_df.index.name = 'System'
display(autoskzcam_hads_terms_df)
autoskzcam_hads_terms_df.columns = [r'$E_\textrm{int}^\textrm{SKZCAM}$', r'$E_\textrm{rlx}^\textrm{DFT}$', r'$E_\textrm{coh}^\textrm{CCSD(T)}$', r'$E_\textrm{conf}^\textrm{CCSD(T)}$', r'$\epsilon_\textrm{geom}$', r'$E_\textrm{ads}^\textrm{autoSKZCAM}$', r'$\Delta H^\textrm{DFT}$', r'$\Delta H^\textrm{autoSKZCAM}$']
autoskzcam_hads_terms_df.columns.name = 'Quantity'

convert_df_to_latex_input(
    autoskzcam_hads_terms_df,
    start_input = '\\begin{table}',
    label = 'tab:autoskzcam_hads_terms',
    caption = r"A summary of the terms which make up the final autoSKZCAM $E_\textrm{ads}$ and $H_\textrm{ads}$ estimate. We show the individual terms which make up these final $E_\textrm{ads}$ values, with the final $E_\textrm{ads}$ value being the sum of the $E_\textrm{int}^\textrm{SKZCAM}$, $E_\textrm{rlx}^\textrm{DFT}$, $E_\textrm{coh}^\textrm{CCSD(T)}$ and $E_\textrm{conf}^\textrm{CCSD(T)}$ values, wherever the last two terms are needed. The errors due to $E_\textrm{rlx}^\textrm{DFT}$ and from using the revPBE-D4 geometry for all the $E_\textrm{ads}$ terms is encapsulated in $\epsilon_\textrm{geom}$ from an ensemble of DFT XC functionals. An additional $\Delta$H term is also calculated, further calculated from a DFT XC functional ensemble.",
    end_input = '\\end{table}',
    filename = 'Tables/SI_Table-AutoSKZCAM_Hads_Terms.tex',
    adjustbox = 0.9,
    center = True,
    df_latex_skip = 0,
    rotate_column_header = True
)


,SKZCAM Eint,DFT Erlx,CCSD(T) Ecoh,CCSD(T) Econf,epsilon geom,autoSKZCAM Eads,DFT DeltaH,Final
System,,,,,,,,
\ce{CH4} on MgO (001),-120 ± 2,2,0,0,5,-118 ± 6,11 ± 13,-107 ± 14
Monolayer \ce{CH4} on MgO (001),-119 ± 3,12,-36,0,11,-143 ± 11,13 ± 14,-130 ± 18
\ce{C2H6} on MgO (001),-170 ± 4,1,0,0,10,-169 ± 10,-1 ± 10,-170 ± 14
Monolayer \ce{C2H6} on MgO (001),-156 ± 3,16,-73,0,17,-213 ± 17,15 ± 14,-198 ± 22
\ce{CO} on MgO (001),-204 ± 4,8,0,0,7,-195 ± 8,19 ± 3,-176 ± 8
\ce{C6H6} on MgO (001),-415 ± 9,26,0,0,38,-388 ± 39,-10 ± 9,-399 ± 40
Parallel \ce{N2O} on MgO (001),-249 ± 3,3,0,0,4,-245 ± 5,-6 ± 1,-251 ± 5
Tilted \ce{N2O} on MgO (001),-164 ± 4,10,0,0,6,-154 ± 7,-4 ± 4,-158 ± 8
Vertical-Hollow \ce{NO} on MgO (001),35 ± 4,29,0,0,43,64 ± 43,6 ± 6,71 ± 44


# Insights into molecular binding

### CO<sub>2</sub> on MgO (001)

### H<sub>2</sub>O and CH<sub>3</sub>OH on MgO (001)

### NO on MgO (001)

### N<sub>2</sub>O on MgO (001)

### CO<sub>2</sub> on TiO<sub>2</sub> rutile (110)

# Comparison between theory and/or experiment

### *H*<sub>ads</sub> comparison between autoSKZCAM and experiment

In [112]:
expt_hads_df

,Surface,Molecule,Temperature,log(ν),Eact,Hads,Error,Details
0,MgO,CH4,47,13.1 ± 2.0,-115,-113,19,Dilute limit $E_\textrm{act$ estimate by Tait ...
1,MgO,C2H6,75,14.9 ± 2.0,-221,-218,30,Dilute limit $E_\textrm{act$ estimate by Tait ...
2,MgO,CO,61,13.8 ± 1.6,-,-179,19,Average of the $H_\textrm{ads$ re-analysis by ...
3,MgO,N2O,77,13.0 -> 14.0 ± 2.0,-230,-242,31,$E_\textrm{act$ measured by Lian \etal{~\cite{...
4,MgO,C6H6,162,15.1 ± 1.6,-,-488,68,Average taken of $H_\textrm{ads$ re-analyzed b...
5,MgO,H2O,203,-,-,-520,121,$H_\textrm{ads$ from Ferry \etal{~\cite{alessi...
6,MgO,NH3,160,13.0 -> 14.0 ± 2.0,-581,-606,63,$E_\textrm{act$ measurement from Arthur \etal{...
7,MgO,Physisorbed CO2,120,13.0 -> 14.0 ± 2.0,-408,-426,48,$E_\textrm{act$ measurement from Meixner \etal...
8,MgO,Chemisorbed CO2,230,13.0 -> 14.0 ± 2.0,-619,-654,123,$E_\textrm{act$ measurement from Chakradhar an...
9,MgO,Monolayer CH4,47,13.1 ± 2.0,-131,-129,19,Monolayer $E_\textrm{act$ estimate by Tait \et...


### *E*<sub>int</sub> comparison between autoSKZCAM and DFT / RPA

# Miscellaneous

### Plot schematic TPD curve

In [16]:
# Plot the TPD curves

# Pandas DataFrame containing the TPD data
tpd_schematic_df = pd.read_csv("Data/Miscellaneous/TPD_schematic_data.csv")

# Make the header a row then set new header  'Temperature (K)' followed by 0.1 to 1.0 in 0.1 increments
tpd_schematic_df.columns = ['Temperature (K)'] + [f'{i:.1f}' for i in np.arange(0.1, 1.1, 0.1)]

# Plot each of the columns with the first column as x and the rest as y
fig, axs = plt.subplots(figsize=(3.365, 2.5),dpi=450,constrained_layout=True)
for column in tpd_schematic_df.columns[1:]:
    axs.plot(tpd_schematic_df['Temperature (K)'], tpd_schematic_df[column], label=r'$\theta{=}$' + column)

axs.legend(loc='upper left',fontsize=8)
axs.set_xlabel('Temperature (K)')
axs.set_ylabel(r'Desorption rate ${-}d\theta/dt$ (ML/s)')

axs.set_xlim([55,76])
axs.set_yticks([])
axs.set_xticks([70])
axs.set_xticklabels([r'T$_{p}$'])

plt.savefig('Figures/Figure_SI-TPD_Schematic.png')
